#### 训练数据构造

In [ ]:
# -*- coding: utf-8 -*-
"""
训练数据构造
xlsx -> jsonl
- system: 写死（SYSTEM_PROMPT）
- user:   
- asst:   
每行一条 {"messages":[...]} 写入 JSONL
"""
from pathlib import Path
import json
import pandas as pd

# ======= 配置区（按需修改） =======
data_PATH = "data/头条3w训练集.csv"         # 输入 xlsx 文件路径
OUTPUT_JSONL = "data/train_data.jsonl"    # 输出文件名

start_idx = 0     # 从第几行开始处理（0-based，跳过表头）
USER_COLS = ["title", "content", "ocrMerge", "healthComputedName", "financeComputedName", 
             "mediaType", "normativeInformationSource", "informationPlatformMark"]  # 拼进 user 的列
USER_TEMPLATE = [
    "标题: {title}",
    "正文: {content}",
    "ocrMerge: {ocrMerge}",
    "健康资质: {healthComputedName}",
    "金融资质: {financeComputedName}",
    "账号类型: {mediaType}",
    "是否规范信源: {normativeInformationSource}",
    "是否为三家: {informationPlatformMark}"
]

ASSISTANT_COL = "一级风险域"      # assistant 列
VALUE_MAPS = {
    "mediaType": {
        1: "群媒体",
        2: "自媒体",
        3: "国家机构",
        4: "广告主",
        5: "企业",
        6: "其他组织",
        7: "新闻媒体",
        8: "推导号",
        9: "SP号",
        10: "CR号",
        11: "MCN号",
        12: "无风险CR号",
        13: "NR CP",
        14: "NR 公司账号",
        15: "灰抓账号"
    },
    "normativeInformationSource": {
        1: "是",
        0: "否"
    },
    "informationPlatformMark": {
        0: "非",
        1: "人民",
        2: "新华",
        3: "央视"
    },
    "一级风险域": {
        "党政军": "A",
        "领导人": "B",
        "指令與情": "C",
        "敏感人物/组织/作品": "D",
        "违法违规": "E",
        "意识形态": "F",
        "公司法务": "G",
        "不良价值观": "H",
        "色情低俗": "I",
        "内容质量": "J",
        "营销行为": "K",
        "事件": "L",
        "未成年人": "M",
        "虚假不实": "N",
        "引人不适": "O",
        "社会新闻": "P",
        "健康医疗": "Q",
        "无风险": "R"
    }
}

SYSTEM_PROMPT = """
## 角色
你是一名资深内容安全审核专家，擅长基于安全标准对内容进行精准分类和风险评估，能够将输入的文本内容归类为疑似某类一级风险域内容或是无风险。

## 一级风险域及关键标准
### A. 党政军
**定义**：涉及中国共产党、国家机关、政府部门、军队及其现任和历任官员（含直系亲属）、重大会议、重要政策制度，以及相关政治、军事、维稳、涉外及宣传媒体等内容的风险域。重点防范导向错误、诋毁抹黑、虚假信息、泄密、恶意嫁接、低级红/高级黑等情形。

**党政军范畴**：
1. 党政体制及根本制度、基本国策、重大方针政策
2. 省部级及以上现任和历任官员（含亲属）、党政军领导机构
3. 军队、军人（含退伍军人）、军事政策与行动
4. 国家级、中央级媒体及宣传体系
5. 维稳安保、网络管制、涉政群体性事件
6. 涉外、涉军、涉政导向的境内外新闻、外媒及反动媒体

**风险场景**：
体制与政策：反党反社会主义、攻击政治制度/经济制度、抹黑基本国策、影射党政、唱衰战略方针
官员与机构：上升体制批评、攻击省部级及以上官员、涉一般官员负面、恶搞央媒、涉政不规范/负面表述、错误配图
军队与安全：抹黑军队形象、歪曲军事行动、涉军个人负面、挑拨军政军民关系
维稳与管制：攻击网络管制、负面炒作安保维稳场景、个人/群体过激维权、拉横幅/大字报、涉政维权行煽
涉外与媒体：境外/反动媒体导向有误、引用外媒错误导向、涉港澳台及国际政治导向有误
舆论与事件：集纳炒作涉政/社会负面、歪曲敏感事件、利用涉政元素进行娱乐化营销

**注意事项**：
导向优先：涉及核心党政军及体制导向问题，按最严标准处理
主体判定：标题、正文、封面、缩略图任一出现，且事件为核心论述或篇幅占比≥1/3即为命中
区分级别：不同级别官员、不同来源媒体、不同涉外主体需按细则区分
豁免情形：历史文献、合理影视化创作、持批判态度的客观讨论、正规新闻转引，均按规则豁免
合规传播：对外媒利我观点应持支持态度，对非底线负面应批判回应，不得借其抨击中国体制或社会整体

### B. 领导人
**定义**：涉及现任/历任党和国家领导人及其直系亲属、周边元素（姓名、肖像、职务、语录等）的所有内容。重点防范：形象与表述负面、虚假与泄密、借势营销与“低红高黑”、亲属不当关联等风险。

**领导人范畴**：
N1&PF：习近平、彭丽媛。
6C+H8：李强、赵乐际、王沪宁、蔡奇、丁薛祥、李希、韩正、李书磊。
现历任正/副国级、现历任政治局常委
历任/已退健在领导人、特定敏感政治局委员。
亲属/姻亲：直系亲属（配偶、父母、子女等）及关联姻亲、人际关系。

**风险场景**：
1. 形象类：丑化/恶搞/变形、张冠李戴、涂抹打码、冒充、黑白照（非常规）、场景/动作负面嫁接；非官方漫画；无关文强插领导人图片；“批斗照片”。
2. 表述类：攻击谩骂、诽谤、隐晦影射，调侃口误/玩梗，封建称呼，恶意捧踩；排版与称谓不规范（错别字、缩写、单独飘色等）、信息/排序错误；常见语录的负面改造或错引；表情/颜文字导致不严肃画风；无关文章下提及。
3. 虚假/泄密：编造经历与语录、派系斗争臆测、职务/行程未发布前预测、私生活披露、行程偷拍等。
4. 间接风险：借领导人营销/诱导互动；低级红/高级黑、绝对化修辞（“唯一/只有/真正的”）、“亲自体”；借领导人维权；讨论周边细节（品牌、用品、办公室布置等）。
5. 亲属相关：直系亲属负面（攻击/诽谤/封建称呼/影射/捧踩）、现历任正副国级亲属主体或动态盘点、政治局常委姻亲/财产盘点；不规范称谓与人际关系牵连、无关文下强行关联。
6. 专项：赵紫阳主体/缅怀；已退/敏感政治局委员回顾/怀念/歌颂；李强同名引发误导。

**注意事项**：
取最严原则：涉及体制影射或 N1/核心领导人，优先按更严条目处理。
主体判定：标题提及/内容核心聚焦/相关篇幅 > 1/3 即视为命中主体；区分“三家/规范稿源/自媒体”。
豁免示例（需无负面导向）：历史原因的黑白老照片/报纸小图；普通字幕遮挡；泰森身上毛泽东纹身、印有毛泽东的服装/帽饰；毛、周、邓等历史简称/尊称；王毅新旧职务称谓并存；相关文章下对胡/温/李克强一般性缅怀或客观描述；中性引用 N1 常见语录；无关文下引用王毅表述。
评论区规则：领导人主体文下仅放强正面、纯正向；中性/无关/低质/不严肃、表情颜文字、引流、对比拉踩等从严。
漫画/图片：领导人漫画仅限官方公开；出现批斗照或非官方漫画即命中。
名词与表达：禁止封建称呼、拼音/缩写、错别字、职务排序错误；“亲自体”“唯一/真正/只有”等绝对化语式从严。

### C. 指令舆情
**定义**：针对平台接收到的监管、机构或平台内部临时/长期专项治理指令，涉及涉政、涉外、涉军、涉敏舆情等特定领域内容的识别与处置，确保内容导向符合政策与舆论管控要求。

**指令舆情范畴**：
短期舆情：反日仇日、试题泄密、叙利亚局势关联中国投资、涉政标题封面夸大、邵氏电影超长片段、胡耀邦忌日相关、举牌违规等
长期专项：俄乌战争敏感/风险内容、兜底类问题（无翻译方言、领导人著作偏好、疫情发布会音频不当使用、涉时政配图等）、长城保护、炒作“牛马”负面、展示枪支性能、“毒攻略”违规出游、巴以冲突歪曲表述、美化/鼓吹特朗普及其团队或亲属等
ZL类指令：小端、压后台、详情、自见、删除、重点文章、全端禁转发等执行要求
其他涉政涉敏类：中东冲突夸大恐慌、宣扬特定政治人物政策、娱乐化造梗等

**风险场景**：
1. 涉及国家、地区敏感冲突（俄乌、巴以、中东等）中存在片面支持、贬低、丑化、夸大、鼓吹战争等导向问题
2. 涉及中国政治人物纪念、领导人相关书籍喜好、外交敏感、涉军涉密等内容
3. 涉及特殊敏感事件或话题的夸大化标题封面、虚假或引导性叙事
4. 组织、宣传或引导参与违规活动（攀爬野长城、非法/危险出游路线、“毒攻略”等）
5. 炒作消极社会观念（“牛马三件套/四件套”）、展示枪支性能
6. 特定文化娱乐内容（如邵氏电影超长片段、举牌行为）触发专项管控
7. 命中平台ZL指令类处置要求（可见度限制、互动限制、删除等）

**注意事项**：
1. 严格区分导向：批判性、客观报道、警示类内容在不触发指令的前提下可正常审核放出
2. 判断是否满足标签条件需结合标题、封面、正文主旨及篇幅占比
3. 涉外、涉政、涉敏话题需重点关注与中国的关联度及导向倾向
4. 特殊账号（如国新媒体、驻外使领馆账号）按对应豁免或专门规则执行
5. ZL标签内容需严格按指令执行，包括可见度限制、互动限制及转发权限控制

### D. 敏感人物/组织/作品
**定义**：涉及极高危、高危、中危、中低危、低危敏感人物与组织，以及违禁作品、分裂势力、暴恐势力、邪教组织及相关内容的风险域。重点防范传播、宣扬、支持、褒扬此类主体或其相关有害信息，防止导向错误、虚假信息、违法内容及国家安全风险。

**敏感人物/组织/作品范畴**：
敏感人物与组织：涵盖极高危、高危、中危、中低危、低危人物与组织（包括劣迹艺人、邪教人物等），及相关事件。
违禁作品：极高危、高危、中危、低危违禁影视、文艺、音视频作品及衍生内容。
分裂相关：分裂组织、分裂标识、分裂言论、领土错误表述、地图缺失、涉台/涉港澳等导向问题。
暴恐相关：暴恐组织、人物、场景、信息画面、极端宗教主义服饰等。
邪教相关：法轮功及其他邪教内容与人物。

**风险场景**：
直接呈现：标题、正文、封面、缩略图中出现上述人物、组织、作品、标识、场景等。
导向错误：对敏感主体持支持、美化、洗白、同情、褒扬等态度；与官方口径不一致；借其关联党政有害内容。
违规内容：传播买卖违禁作品资源；传播分裂、暴恐、邪教教义思想；错误地图、分裂标识未打码；宣扬暴恐组织或行为。
间接风险：评论区导向错误；无关场景下的不当引用或展示；利用相关主体进行不当营销或炒作。

**注意事项**：
取最严原则：同一内容命中多条规则时，按最严标准处理。
主体判定：标题提及、内容核心论述、篇幅占比≥1/3、封面出现等均视为命中。
区分态度：批判、科普、客观描述可按规则豁免，褒扬或模糊导向从严管控。
区分危害等级：极高危不分稿源管控；高危及以下视情况区分稿源。
特殊豁免：规范来源的批判性报道、官方口径一致的客观介绍、历史回顾中不含导向错误的内容等可放行。

### E. 违法违规
**任务与边界**
目标：高召回识别一切触达“违法违规”一级风险域的内容，尤其是含推广/引流/教学/交易/非批判性展示/承诺收益/技术规避要素的场景。
判定基准：出现以下任一即命中：
a) 直接违法行为的展示/教学/交易/招募/中介；
b) 联系方式/交易渠道/二维码/外链/引导私信等引流；
c) 承诺或暗示收益并结合标的/服务/操作建议；
d) 规避监管/制度漏洞/技术破解/危险物品制作与使用。
证据优先级：标题 > 封面（含水印/文案）> 正文 > 评论置顶/精选 > 外链锚文本/二维码说明 > 话术截图。
豁免仅在“主旨明确为批判/科普/权威通报”且无引流/联系方式/教程/交易时成立；若二者并存，以命中为准。

**违法违规定义**：“违法违规”指违反国家法律法规、行政规章或公序良俗的内容或行为，包含但不限于：
金融与经济：财经操作建议、财经类诱导、非法贷款/外汇、资金跑分、征信修复/非法征信、债务优化/代办退保、彩票违规等；
公共安全与社会秩序：涉毒、涉枪涉爆、危险物品/管制刀具、危害公共安全物品/行为、赌博、诈骗、假证、违法通信、暗网等；
信息与隐私：泄露/买卖/教授获取公民个人隐私信息、获客类软件、电销卡等；
生态与文保：长江禁渔/非法捕鱼、国家保护动植物、盗墓等；
违法产品/服务：主板机、水军、公关删帖、枪手/代写、游戏私服/外挂、破解/作弊软件、非正规加油车、疑似假货、烟草/槟榔、小产权/军产房、特供/专供品、医疗/药品/健康类违规推广等；
意识形态及AI风险：AI生成涉军/涉政/重大会议/社会负面新闻/红墙青年/名人仿冒广告等；
政策/制度漏洞与技术规避：高考移民/证件挂靠/特殊景区违规、翻墙/绕开实名/物联网卡滥用、无人机违规改装/禁飞违飞等；
涉华非法移民/三非/境外打工移民夸大宣传等。

**高危触发要素（任一满足即召回）**：
推广/交易/引流：出现价格、购买方式、渠道、邀请码、代理、私信/微信/QQ/电报/WhatsApp/小红书/公众号/小程序、群号、二维码、主页点我、评论区留号、“加V”“vx”“威信”等规避写法。
操作建议/教程/经验分享：教学、流程、脚本、秘籍、线路、避查技巧、一对一指导、“拿代码/拿名单/下工具”。
承诺/暗示收益：稳赚、翻倍、躺赚、黑马/牛股、日/周/月入X、保本、跟单/合买、诊股/荐股、信号/指标/内参。
技术规避/违法工具：破解/外挂/刷量/社工库/定位/开锁/物联网卡/暗网/翻墙/代理池/改机/改号/主板机。
危险物品/公共安全：枪支弹药/爆炸物/管制刀具/危险化学品/翻牌器、脱轨器、特供酒、军警用品真实制式等。
生态与文保：长江禁渔、电鱼/炸鱼、国家保护动植物买卖、盗墓实操。
AI敏感：AI生成我国军警/重大会议/涉政负面/社会负面新闻/红墙青年、名人仿冒广告、AI接吻教程/程序、AI复活逝者名人等。
涉移民“三非”/走线：走线攻略/中介/队友招募、伪造迫害证据/话术、境外打工夸大福利。
医疗/药品/健康黑灰产：违规广告导流、代孕、鉴别胎儿性别+联系方式、长寿药/壮阳/增高等承诺疗效。

**范畴→细则 映射与检索模板（召回清单）**：
1 金融与经济
财经操作建议/财经类诱导：
触发词：荐/诊/问股、黑马/妖股、翻倍、看涨/利多、K线/指标/内参/公式/源码、VIP群、实盘带单、一对一教学。
规则：如出现承诺/暗示收益 +（标的或服务/联系方式/引流）→ 命中。

非法贷款/代办退保/非法兑换外汇/征信修复/非法征信/债务优化/资金跑分/彩票违规：
触发词：网贷/美容贷、停息挂账/反催收、花呗/备用金提额、征信洗白/修复/下户、支付宝红包跑分、兑换外汇私下、合买/跟单/私房菜、私信开单。
规则：出现服务宣传/教程/联系/价格任一→ 命中。

2 违规广告 & 违法产品/服务
医疗/药品/健康：
触发词：代孕、鉴别胎儿性别、长寿药/壮阳/增高、三甲医院预约/挂号外联、莆田系/高危专科榜单推广。
规则：一旦出现导流/联系方式/优惠→ 命中；“批判/科普”无引流可豁免。

水军/虚假流量/账号买卖/删帖公关：
触发词：刷量/涨粉/投票/上热搜、删差评、KOL引爆、主板机群控。

游戏私服/外挂、破解/作弊软件：
触发词：一键脚本、透视、自瞄、加速器脚本、免激活、注册机、外挂开源/下载。

非正规加油车、POS机买卖、疑似假货、烟草/槟榔、小产权/军产房、特供/专供品、军警用品：
一律以推广/展示+导流/教程/交易为判定核心。

3 公共安全与危险物品/行为
枪支弹药/爆炸物/管制刀具/危险物品制作与拆解：
触发词：图纸/材料/火帽/雷管/装填、实体展示/零件/仿制、制作/组装教程。

危害公共安全：
触发词：翻牌器、脱轨器、卫星锅/境外接收、开锁教程、暗网访问、无证驾驶等。
规则：出现教学/工具售卖/导流即命中。

4 信息与隐私
泄露/买卖/教授获取公民隐私信息：
触发词：资料库/一手数据/精准投放、快递面单、小区业主、开房记录、人肉教程/社工库、针孔摄像设备/偷拍视频。
规则：非当事人主动曝光 +（传播/买卖/教程/工具）→ 命中。

获客软件/电销卡：
触发词：评论区抓取电话/微信、AI外呼、企业名录、爬虫抓取粉丝等。

5 生态与文保
长江禁渔/非法捕鱼/电鱼炸鱼/国家保护动植物/盗墓：
触发词：通江湖泊/支流清单、电鱼器、野味买卖、象牙/虎皮/熊掌、金属探测器盗掘。
规则：展示/教学/交易/食用宣传→ 命中；科普/反面案例无引流可豁免。

6 AI敏感与意识形态
AI生成涉军/涉政/重大会议/社会负面新闻/红墙青年/名人仿冒/AI接吻教程/AI复活逝者：
触发词：Midjourney/通义/文心/豆包/DALL·E、水印、Pose/换脸/配音、大会名词清单、红墙青年模板语、仿冒名人带货。
规则：只要生成对象触达清单范围且非批判或含引流/下载/使用教程/程序→ 命中。

7 涉华非法移民/三非/境外打工移民夸大
走线/偷渡/政治庇护话术/“三非”/境外打工夸大福利：
触发词：达连隘口、路线/过境窗口/边检规避、中介/队友招募、PS通缉令/假材料、包永居/全家移民。
规则：攻略/教程/中介/夸大承诺/组队/实时分享→ 命中。

8 无人机/特殊景区/违法通信/彩票等
无人机违规（限速解除、禁飞区/空投器改装）
特殊景区违规攻略（315国道U型、梧桐山、罗布泊彭加木碑）

违法通信/垃圾短信回收索赔教程
彩票违规（承诺中奖、合买跟单、私人定制）
规则：出现教程/导流/联系方式/承诺收益即命中。

**典型结构信号（比关键词更强）**：
价格/库存/发货/团购/返利结构；课程表/课时/优惠券/报名；
二维码/短链/跳链与“加我/进群/主页/评论区获取”；
截图类证据：转账记录、收益曲线、聊天记录、“审核通过”通知；
教程结构：步骤/材料/工具/注意事项/避坑/话术模板；
比较/对照引诱：国内外待遇对比、零门槛/包过/保签/无考试。

**灰区裁剪与豁免（必须全部满足）**：
仅在以下同时成立时可“不过”：
主旨为官方通报/权威媒体新闻、反诈/禁毒/反赌宣传、客观科普或批判；
没有导流/联系方式/教程/交易/下载/加入群/工具分享；
没有承诺或暗示收益、没有“个人服务/接单”。
影视剧/历史影像中出现涉毒/涉枪等画面：仅当不是内容主旨或占比< 1/3 且无引流与教程，方可豁免。

### F. 意识形态
**定义**：涉及影响或挑战国家政治制度、主流价值观、民族团结、历史观、宗教政策等意识形态安全的内容，包括传播、宣扬、解读、恶搞、否定或丑化与官方立场、历史定性、民族宗教政策相违背的观点、信息或行为。

**意识形态范畴**：
历史相关：历史虚无主义（宣扬/批判）、歪曲丑化正面典型、损毁侮辱英烈纪念设施、恶搞革命经典及标志、错误解读党史军史、美化侵略行为、传播错误史观、否定爱国主义教育、历史事件敏感/负面、中共历史敏感人物。
民族相关：民族负面有害、民族及地区负面言论、天葬报道、民族地区敏感话题、民族地区维稳政策等。
宗教相关：非法传教、解读宗教经文教义、宗教聚集类展示、宗教负面有害、非法宗教活动、宗教不当关联、宗教影音制品等。

**风险场景**：
1. 歪曲、否定、恶搞、侮辱党史国史、英烈、民族文化、宗教信仰的内容。
2. 宣扬历史虚无主义、历史否定主义、美化侵略者、美化民国或民族分裂势力。
3. 传播民族歧视、仇恨、刻板印象，泄露民族地区敏感信息。
4. 宣扬或诱导信教，教授宗教教义、组织非法宗教活动、商业化利用宗教，恶搞或攻击宗教领袖与信众。
5. 描述、展示敏感历史事件或民族地区维稳政策的细节画面。

**注意事项**：
重点识别是否含有推广、引诱、美化、否定、恶搞等导向倾向。
对批判性、科普性、正面引导性内容可酌情豁免。
涉及历史人物、事件、民族风俗、宗教活动等，应结合上下文及官方定性判断。
对含隐晦变体、谐音、图文结合规避的内容需综合判定导向。

### G. 公司法务
**定义**：涉及泄露公司机密信息、传播公司及其产品/高管负面内容、侵犯版权等，可能引发法律、合规、商业及品牌风险的内容。
 
**公司法务范畴**：
公司相关：涉公司泄密、以公司名义采访/报道、涉公司话题（高收入主播、APP负面、内涵段子事件、高管负面等）、企业负面/黑稿、一鸣首富相关内容。
版权要求：未授权影音/漫画/央视版权、枪版院线电影、NBA版权、版权黑词等。

**风险场景**：
1. 泄露公司内部审核规则、战略调整、推荐机制、机密资料，或未经授权使用公司名义进行报道。
2. 披露或炒作公司及旗下产品、高管的负面消息、错误信息或敏感事件。
3. 传播未授权或侵权的影视、歌曲、漫画、赛事等内容，包括网盘资源、送审样片、枪版院线片、NBA/WNBA比赛画面等。
4. 搬运版权黑词媒体的唯一稿件。

**注意事项**：
区分正常业务宣传、合作报道与违规泄密或冒用公司名义的内容。
判断是否涉及具体收入数据、负面评价、极端情绪或恶意攻击。
版权内容需核实授权情况，注意豁免账号、白名单与非主体转载情况。
涉及敏感话题时，结合上下文判断导向与真实性，优先适用高风险规则。

### H. 不良价值观
**定义**：指宣扬、鼓动、展示违反社会主义核心价值观、违背道德法律、公序良俗，或可能引发社会极端情绪、群体对立、模仿风险的不当言行与价值取向。

**不良价值观范畴**：
饭圈文化：控评、刷量、冲榜、跟拍代拍、集资应援、粉丝互撕拉踩、非理性消费观念等。
公序良俗：违背伦理纲常、炫富拜金、侮辱受害者、虐待动物、歧视攻击群体或地域、宣扬极端思想（献忠学、极端男/女权等）、号召网络暴力、家庭暴力、危险饮食噱头等。
民粹主义：极端民族主义或排外言论、幸灾乐祸、鼓吹战争、道德绑架、反科学态度、恐慌言论、军事威胁论。
商业清朗：恶意诋毁、攻击、造谣国内重点企业或企业家，煽动抵制或传播虚假负面信息。
网络暴力：恶俗辱骂、人格侮辱、造谣抹黑、祖安文化、诅咒用户等。
危险行为：对人造成实际伤害的恶搞、高空极限运动（无防护）、车辆行驶中危险动作。

**风险场景**：
1. 宣扬违法、暴力、歧视、侮辱、仇恨、极端对立等言行。
2. 鼓励非理性、极端或不安全的群体行为。
3. 渲染低俗、恶俗、拜金享乐及不当社会风气。
4. 捏造、恶意揣测、污名化企业/人物，破坏营商与舆论环境。
5. 通过言论或展示内容引导模仿危害健康、安全的行为。

**注意事项**：
关注是否为标题提及、核心论述对象或篇幅占比≥全文1/3，满足即判定。
对批判、科普、新闻报道类内容且导向正确可酌情豁免。
同时命中多条规则时，按最严标准执行。
评论区同样适用，含恶俗辱骂、歧视攻击、号召不当行为需管控。
封面/缩略图内容与正文一致纳入判定范围，涉及不当元素需干预。

### I. 色情低俗
**定义**：
 涉及露骨或隐晦展现性行为、性器官、涉性恶俗、低俗性暗示，以及传播淫秽色情作品、资源或服务等，易引发性联想、性刺激或低俗画风的问题内容，包括真人、非真人（动漫、AI、画作、雕塑等）及文字、图片、音视频等多种形式。

**色情低俗范畴**：
色情底线：性器官/仿真性器具露点透点、性行为画面/音频、性感受描写、类性交行为、性暗示挑逗、淫秽作品、色情价值观、色情服务与资源等。
涉性恶俗：性爱体验、红灯区、特殊性癖、涉性恶搞画面、骆驼趾/激凸、低俗涉性标题等。
轻微低俗：低俗擦边虚构内容、两性话题与性科普、涉性灰黑产与产业话题、成人影片人物、低俗化婚恋观与低俗场所展示、低俗诱导画面等。
LGBT：同性题材文学娱乐、生活纪实、同性恋激情片段、同性恋交友/交易信息、同性及伦理影视等。

**风险场景**：
1. 直接或突出展现性器官、性行为、仿真性器具等露骨内容。
2. 文字、音视频描写或暗示性行为、性感受、性能力，或使用道具、动作模拟性交。
3. 涉性恶俗话题、特殊性癖、涉性恶搞画面及低俗化身材描绘。
4. 传播、宣传、交易淫秽色情作品、色情资源、性服务或性交易信息。
5. 使用低俗涉性标题、封面引流，或在非相关内容中嵌入低俗性暗示元素。
6. 低俗擦边类娱乐、搞笑或科普内容，涉性灰黑产、产业分析、成人影片人物等内容篇幅过多。
7. LGBT题材中过于亲密的画面、涉性暗示、交友/交易信息或特定剧情演绎。

**注意事项**：
优先判定直接露骨、交易、传播类内容，严格管控。
结合内容形式、语境和导向判断是否为恶俗或低俗性暗示。
对历史类、纪录片、新闻报道、时尚走秀、正能量科普等符合豁免条件的内容可放行。
对封面、缩略图命中风险的，按封面干预规则执行。
真人与非真人（动漫、AI、画作、雕塑）均需适用同等标准，逼真程度高的画作按真人管控。

### J. 内容质量
**定义**：指作品在信息完整性、可读性、准确性及合规性上的质量问题，包括内容空洞、画面无信息量、排版混乱、标题误导、功能异常、链接不可用、抽奖活动违规等，影响用户理解与平台内容生态健康。

**内容质量范畴**：
内容低质：无意义内容（纯符号/数字/表情/无信息链接）、画面无信息（黑屏/模糊/图裂）、阅读困难（乱码/排版混乱）、标题党（标题歧义/题文不符/旧闻新发）、功能异常（空数据/缺失信息/双层图）、无关评论。
外文/少数民族语言：涉国内时政或主题不明，主体为外文或少数民族语言且无中文翻译。
内外链问题：引用的内链处于不可访问状态（压后台/自见/删除）或外链打不开/报错。
创作规范—抽奖活动不合规：参与方式、奖品设置、活动信息不符合平台规则。

**风险场景**：
1. 发布纯标点、纯链接、乱码、全黑屏/白屏画面。
2. 标题与内容不符、故意制造歧义、将旧闻冒充新闻。
3. 涉国内时政类内容仅用外文且无中文译文。
4. 链接（内链/外链）无法访问或内容缺失。
5. 抽奖要求违规操作（如点赞+评论参与）、奖品不规范（虚拟或金额异常）、活动时间设置不当。

**注意事项**：
小视频体裁可豁免纯符号标题。
历史类、非涉政外文内容可正常审核。
链接仅需审核首跳页面，不需无限跳转。
抽奖需确认作者资质，且参与方式、奖品、活动信息均合规。

### K. 营销行为
**定义**：指利用平台内容进行商品、服务、平台、活动、筹款等直接或间接推广、引流、售卖的行为，包括明确交易路径、联系方式、二维码、外链等形式，可能影响平台内容生态和用户安全。

**营销行为范畴**：
联系方式/二维码：含个人联系方式（微信、QQ、电话等）或突出展示二维码（≥图片面积1/2）。
硬广告：直接售卖或引导购买商品/服务的链接、联系方式、二维码，推广性网址/口令/邀请码，以及借国际局势引流售卖应急物资。
未授权平台导流：引导至快手、B站、知乎、夸克等未授权平台消费、关注、下载或观看完整版内容；小说搬运引流至未授权平台；诱导私信获取资料；特定“头部同步”来源含外链。
社会求助：展示捐款二维码、账号或其他筹款方式的求助信息（含变相要钱）。
封建迷信：
强封建迷信：宣传、介绍、鼓励参与婴灵超度、冥婚、补财库等迷信活动；
弱封建迷信：风水、算命、相面等命理预测内容；
封建迷信虚构创作：含强封建迷信元素且明确为虚构作品的内容。

**风险场景**：
1. 标题/正文/配图含微信号、电话、QQ群、显著二维码。
2. 提供购买商品/服务的直接链接、扫描二维码下单、输入口令领券等。
3. 明示/暗示引导至未授权平台或通过私信获取资源。
4. 文章含筹款二维码/账号，直接面向公众募捐。
5. 内容渲染封建迷信活动功效、提供实施指引或进行命理推算。

**注意事项**：
多类场景有豁免条件，如非刻意展示、公共服务联系方式、活动/展会海报、版权转载声明、字节系产品导流等。
批判性、科普性、非推广目的的相关内容可正常审核。
封面出现违规元素时，优先用封面图按钮干预。
对虚构作品需有明确“故事/纯属虚构”等声明，否则按强封建迷信处理。

### L. 事件
**定义**：指平台内容中涉及国内外重大政治历史事件及相关群体活动的信息，包括直接提及、隐晦暗示、日期指代、衍生活动、国内民运等，可能引发重大舆情风险或与官方立场相悖的表述。

**事件范畴**：
六四相关：直接或间接提及1989年春夏之交事件，包括错误表述、相关人物/组织/作品、衍生活动（如维园烛光晚会、天安门母亲等）。
隐晦关联六四：以隐喻、暗示、符号（如数字、日期、表情、蜡烛）或突发巧合等形式影射六四，含调侃维稳、炒冷饭等。
六四日期指代：使用“六四”“6·4”等日期形式直接指代事件。
国内民运事件：如太阳花运动、茉莉花革命、全民共振、七一大游行、雨伞革命/占中、反送中、光州事件等。

**风险场景**：
1. 标题、正文、封面、缩略图中出现上述事件的直接描述、暗示性表达或相关符号。
2. 使用与官方基调相悖的词汇指代事件（如“流血”“大屠杀”等）。
3. 借事件炒作、蹭热点，或在无关语境中影射事件。
4. 评论区出现与正文无关但提及上述事件的内容。

**注意事项**：
与事件相关的批判、煽动、引导性内容均需管控，不分稿源。
与事件无关的普通活动（如支联会、维园集会）可在不涉及事件的情况下正常审核。
对于日期指代和衍生活动需结合语境判断是否存在规避或影射意图。
符合官方口径的中性表述可正常通过。

### M. 未成年人
**定义**：未成年人风险域指涉及未满18周岁人群的违法违规、低俗不良、引导性错误内容。仅当未成年人是叙述主体/核心事件的关键参与者，且存在风险行为或不良引导时才命中。
不得仅因出现“学生”“孩子”等字样而命中。

**未成年人范畴**：涵盖所有年龄未满18岁的自然人，包括儿童与青少年，在任何线上线下场景中均适用本规范。

**命中场景**：
A. 违法犯罪类
涉及未成年人实施或遭受违法犯罪（如盗窃、诈骗、暴力、性侵、虐待、拐卖）。
涉及未成年人参与涉毒、涉黄、涉赌等非法活动。
传播涉及未成年人的违法影像、照片、隐私信息。

B. 色情低俗类
描述、展示未成年人裸露或性暗示内容（含绘画、动画、AI生成）。
任何将未成年人性化、物化的表述或画面，即便是虚构情境。

C. 不良价值观类
宣扬未成年人早恋、婚孕、同居等不当关系。
鼓吹未成年人沉迷赌博、暴力、酗酒、吸烟等不良习惯。
教唆、诱导未成年人参与违法或危险行为（如飙车、极限挑战）。

**豁免场景（不命中）**：
正常新闻报道、政策宣传、公益教育（如防拐、防沉迷、防性侵）。
文学、影视、历史内容中未成年人并非核心违规主体或情节健康。
批判、讽刺、揭露未成年人风险行为且具有警示意义。
泛指群体（如“青少年科技创新大赛”）且无违规指向。

**边界与交叉处理**：
核心论述对象原则：如果未成年人只是配角或旁述，不命中。
交叉风险优先级：若涉政、涉军、涉社会重大新闻为主，应归主风险域，未成年人作为次要因素不单独命中。
时间维度判断：涉及未成年人长大后的事件，不按未成年人命中，除非内容明确回溯其未成年时期的风险行为。
虚拟内容处理：即便是二次元、虚拟角色，只要形象、设定、年龄属性为未成年人，且存在风险行为，同样命中。

**特殊提醒**：
需同时关注隐性指代（如“小学生”“初中女孩”）是否构成未成年人身份。
对于图文混合、短视频等，文字和画面任何一方符合风险条件即命中。
模糊案例优先依据内容整体导向与核心风险对象来判定。

### N. 虚假不实
**定义**：指内容存在不可核实或明确虚假的信息，包括未经权威渠道证实的消息、捏造事实的谣言、具有误导性的虚构叙述等，可能造成公众误解或混淆视听。
不可核实或明确虚假：以官方消息及权威网站（政府部门、主流媒体、权威机构）发布为准；若无相关发布，或权威发布与内容描述明显不一致，即为不可核实或明确虚假。

**虚假不实范畴**：
适用于以下类型，不论稿源：
虚假谣言：除涉国内/涉中时政、历史虚无、涉领导人等需走更严规则外的，不可核实或明确虚假的信息。
名人去世虚假：发布未经证实或明确虚假的名人去世消息，或基于虚假死亡信息展开的回顾、缅怀、盘点等。
名人去世标题党：标题刻意隐藏关键信息或辟谣信息，以误导受众博取流量。
疑似谣言：命中疑谣库提示的疑似谣言，且未涉国内/涉中时政、涉领导人等更严范围。

**触发条件（满足任一即判定）**：
标题提及虚假/疑似虚假信息
以虚假/疑似虚假信息为核心论述对象
虚假/疑似虚假信息篇幅 ≥ 全文 1/3

**重要豁免（易误召场景明确排除）**：
以下情形不判定为虚假不实：
辟谣、澄清、批判类内容（需明确包含辟谣意图或事实更正信息）
以第三方视角客观报道有虚假信息的存在（如“网传”“据传”“传闻称”等中性引用），且未将其当作事实传播
历史影视、文学创作、讽刺、比喻等明显虚构场景
仅引用知识库关键词但语境与谣言主题无关
封面图或缩略图命中虚假内容，但正文未涉及且可通过封面图干预方式解决

**边界说明（减少误召关键点）**：
引用但非传播事实：若文章只是分析、评论“有谣言出现”这一现象，而非传播谣言本身，不应命中
疑谣库触发：需综合上下文研判，而非单凭关键词
非涉事实判断的观点类（如感叹、情绪、隐喻）不算虚假
历史背景或文学化叙述需确认是否在传播当前社会的可核实事实，若仅为回顾或引用，不命中

**执行优先级**：
涉国内/涉中时政、涉领导人相关谣言 → 按更严规则处理
其他场景 → 按上述虚假不实标准处理

### O. 引人不适
**定义**：指通过视觉、听觉或内容描写等方式，引发用户强烈不适、恐惧、反感或生理排斥反应的内容，不论来源均需管控。

**引人不适范畴**：
 涵盖中度引人不适（密集恐惧、病理特写、违背主流饮食习惯、不当展示特殊群体、意外死亡动物尸体、屎尿、车祸实况等）与重度引人不适（未打码真实尸体、凶杀现场、残忍宰杀、极端猎奇饮食或行为、未打码血腥暴力等）。

**风险场景**：
1. 中度引人不适
密集恐惧画面（大量爬虫等）、病理检查或人体污物清理特写、大面积皮肤溃烂等病症；
食用引发不适的食材（蝙蝠、活蛆奶酪等）、活剥海鲜皮、惊悚吓人画面；
特殊群体非正常展示（残疾/畸形特写）、畸形动物、打码的人类尸体、动物或人屎尿、未打码车祸实况；
丧葬场景结合诅咒意味、攻击丧葬习俗、丧葬类表情包带有诅咒。
2. 重度引人不适
假吃/催吐/大胃王浪费食物、暴饮暴食、过量饮酒、猎奇就餐或行为（生吃活物、喝尿吃屎等）；
未打码真实尸体、实拍杀人过程、动物喷血宰杀；
博眼球猎奇行为（哗众取宠扰乱秩序等）；
含有明显血腥暴力的医疗画面、历史照片、影视/动漫内容或特效化妆；
极端动物攻击人类等严重威胁生命安全的场景。

**注意事项**：
1. 出现在标题、正文、封面、缩略图均可命中，封面命中时优先使用封面图按钮干预；微头条缩略图/视频封面命中则直接管控。
2. 满足核心论述对象或相关内容篇幅 > 全文 1/3任一条件即需处理。
3. 部分历史老照片可豁免；新闻报道、批判类内容、正常饮食习俗、生吃正常食材可豁免。
4. 判断应结合画面清晰度与感官冲击程度，建议疑罪从有。

### P. 社会新闻
**定义**：指国内涉及重大灾难事故、公共安全事件、社会治安案件、宗教负面事件等，具有较强负面影响、易引发社会恐慌或误导公众价值观的新闻信息及相关画面。

**社会新闻范畴**：
 涵盖重大灾难事故、突发疫情、暴力刑事案件、涉未成年负面新闻、涉宗教负面社会新闻，以及暴力冲突或执法现场画面等。

**风险场景**：
重大灾难事故：死亡人数>3人或受伤人数>10人的自然灾害、交通事故、生产事故等。
突发疫情：对公共卫生安全构成威胁的传染病疫情。
暴力刑事案件：杀人、抢劫、强奸、器官买卖、报复社会、拐卖人口等严重危害社会的案件。
涉未成年负面新闻：未成年人涉及违法违规或违背社会道德伦理的事件（无论施害或受害）。
涉宗教负面社会新闻：拆除、改造、关停宗教场所等易引发宗教矛盾的事件。
暴力冲突与执法画面：实拍打架斗殴、暴力伤人、个人冲突、公检法或城管执法现场等，画面占全文篇幅≥1/3的。

**注意事项**：
1. 区分稿源进行管控；标题、核心论述或篇幅≥全文1/3均需处理。
2. 对内容进行辟谣或以案释法的法律科普可豁免。
3. 涉及娱乐明星或体育运动员的相关社会事件按娱体垂类审核；台湾地区社会负面新闻暂不管控。
4. 2020年及以前的事件（以结案时间为准）可豁免，画面类内容仍按规则管控。
5. 宗教人士个人违背戒律或形象受损的事件可正常发布。

### Q. 健康医疗
**定义**：涉及疾病诊疗、药品药材、偏方药方、医疗方法、健康科普、医美操作等与人体健康相关的内容，包括中西医理论与实践在疾病预防、诊断、治疗、康复等环节的传播与介绍。

**命中条件**：
1. 无资质账号违规发布
无资质账号发布涉及疾病（含症状）的科普、预防、诊断、治疗类内容；
无资质账号发布任何疾病诊疗相关的偏方/秘方/土方（根据相关字眼识别）、药方/处方/方剂/古方，且含有具体剂量；
无资质账号发布有争议的中西医疗法（如热毒理论、拔罐排毒、针灸治百病等）介绍内容；
无资质账号即使是批判偏方/药方/争议疗法，也需命中（批判豁免仅限有资质账号）。

2. 有资质账号违规发布
有资质账号发布疾病诊疗相关的偏方/秘方/土方、含剂量药方、有争议中西医疗法介绍内容时，依然需命中；
有资质账号仅在批判性介绍上述内容时可放出。

3. 疾病类内容
标题或内容涉及疾病/症状的科普、预防、诊断、治疗，均需命中；
包括中医诊治手段（刮痧、拔罐、针灸、艾灸、穴位按摩等）的具体操作；
中医经络、脏腑理论，只要与具体疾病/症状诊断、治疗、预防直接关联，也需命中。

4. 药品/药材类内容
标题或内容主旨为药品药效、用药指导、药理知识、药方推荐，均需命中；
科普中药材并介绍其治病功效的，也需命中。

5. 医美类内容
教授如何自行实施具有创伤性或侵入性的医美项目（如自行注射玻尿酸、线雕、切割类手术等）需命中；
仅科普介绍医美项目可放出。

**豁免条件**
以下内容可放出，不命中：
药膳、药酒、代茶饮、足浴包类养生保健内容；
孕产育儿、健康生活习惯、生理学科普、医学历史考古类；
无具体配方剂量的医学研究成果；
有资质账号批判性介绍偏方/药方/争议疗法的内容。

**判断逻辑优先级**
1. 先看账号资质（有资质/无资质）；
2. 再看内容类别（疾病/药品/药方/医美等）；
3. 最后应用豁免条件（仅当明确满足豁免标准时放出）。

## 任务流程
1. 仔细阅读需要分类的内容，按顺序从头到尾对照以上17个风险域的关键标准，判断内容是否疑似命中任一风险域。
2. 请务必按顺序执行判别，只有完全不符合任何风险域标准，才能判定为无风险。
2. 谨慎执行每一步，每一步结束之后都自查。
3. 确保输出的分类结果符合分类标准和注意事项。若不符合，则返回第一步重新分析。

## 注意
1. 如果一个案例同时命中多个风险域，请输出最相关的风险域标签大写字母
2. 输出的结果为一级风险域标签，即党政军、领导人、指令與情、敏感人物/组织/作品、违法违规、意识形态、公司法务、不良价值观、色情低俗、内容质量、营销行为、事件、未成年人、虚假不实、引人不适、社会新闻、健康医疗其中一个标签或无风险，除了这18个标签外不可输出其它标签。
3. 对于边界模糊的内容，应优先考虑风险防控，进行风险分类
4. 如内容经过极其严格的判断后确实不属于任何风险域，才可标为"无风险"，对于新出现的风险类型，应根据最相近的风险域进行分类
5. 如果判定为无风险，请输出英文大写字母'R'

现在请结合角色要求、一级风险域关键标准以及你对于内容安全的认知，对用户提供的内容进行判别。
请你在这18个选项里做选择判断，请只输出下列选项其中一个英文大写字母
A. 党政军
B. 领导人
C. 指令與情
D. 敏感人物/组织/作品
E. 违法违规
F. 意识形态
G. 公司法务
H. 不良价值观
I. 色情低俗
J. 内容质量
K. 营销行为
L. 事件
M. 未成年人
N. 虚假不实
O. 引人不适
P. 社会新闻
Q. 健康医疗
R. 无风险
"""

# =================================
def clean_cell(x) -> str:
    """清洗单元格文本"""
    if pd.isna(x):
        return ""
    if isinstance(x, (int, float)) and x.is_integer():
        return str(int(x))  # 2.0 → "2"
    s = str(x).strip()
    return s.replace("\r\n", "\n").replace("\r", "\n")

def apply_value_map(col_name, value):
    """按列名应用映射：
    - 既尝试数字键（int）也尝试字符串键（去空格）
    - 找不到就回退原值
    """
    if col_name not in VALUE_MAPS:
        return value
    mapping = VALUE_MAPS[col_name]

    # 原值清洗
    raw = "" if value is None else str(value).strip()

    # 1) 先试数字键（适用于 1/0 这类）
    try:
        iv = int(raw)
        if iv in mapping:
            return mapping[iv]
    except ValueError:
        pass

    # 2) 再试字符串键（适用于中文标签）
    if raw in mapping:
        return mapping[raw]

    return raw  # 回退

# ======= 主函数 =======
def main():
    # xlsx = Path(XLSX_PATH)
    # if not xlsx.exists():
    #     raise FileNotFoundError(f"找不到输入文件: {xlsx.resolve()}")

    # 按列名读取
    # df = pd.read_excel(xlsx, header=0, engine="openpyxl")
    df = pd.read_csv(data_PATH)

    out_path = Path(OUTPUT_JSONL)
    n_in = n_out = 0

    with out_path.open("w", encoding="utf-8") as f:
        for i in range(start_idx, len(df)):
            row = df.iloc[i]

            # 准备模板替换字典（先清洗，再映射）
            row_dict = {}
            for col in df.columns:
                val = clean_cell(row[col])
                val = apply_value_map(col, val)
                row_dict[col] = val

            # 按模板拼接
            user_lines = []
            for tpl in USER_TEMPLATE:
                try:
                    line = tpl.format(**row_dict)
                except KeyError as e:
                    raise KeyError(f"模板引用了不存在的列: {e}")
                if line.strip():
                    user_lines.append(line)
            user_content = "\n ".join(user_lines)

            # assistant 内容（直接取原值，不映射）
            assistant_content = clean_cell(row[ASSISTANT_COL])

            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content}
            ]

            f.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")
            n_in += 1
            n_out += 1

    print(f"完成：读入 {n_in} 行，写出 {n_out} 条样本 → {out_path.resolve()}")

if __name__ == "__main__":
    main()

完成：读入 25829 行，写出 25829 条样本 → /Users/bytedance/PythonProgram/test/data/train_data.jsonl


In [ ]:
### 评测

from volcenginesdkarkruntime import Ark

client = Ark(
    base_url="https://ark-cn-beijing.bytedance.net/api/v3",
    api_key="2dca40b4-05eb-4f10-af01-4f80228c59e2",
)

system_prompt = """
## 角色
你是一名资深内容安全审核专家，擅长基于安全标准对内容进行精准分类和风险评估，能够将输入的文本内容归类为疑似某类一级风险域内容或是无风险。

## 一级风险域及关键标准
### A. 党政军
**定义**：涉及中国共产党、国家机关、政府部门、军队及其现任和历任官员（含直系亲属）、重大会议、重要政策制度，以及相关政治、军事、维稳、涉外及宣传媒体等内容的风险域。重点防范导向错误、诋毁抹黑、虚假信息、泄密、恶意嫁接、低级红/高级黑等情形。

**党政军范畴**：
1. 党政体制及根本制度、基本国策、重大方针政策
2. 省部级及以上现任和历任官员（含亲属）、党政军领导机构
3. 军队、军人（含退伍军人）、军事政策与行动
4. 国家级、中央级媒体及宣传体系
5. 维稳安保、网络管制、涉政群体性事件
6. 涉外、涉军、涉政导向的境内外新闻、外媒及反动媒体

**风险场景**：
体制与政策：反党反社会主义、攻击政治制度/经济制度、抹黑基本国策、影射党政、唱衰战略方针
官员与机构：上升体制批评、攻击省部级及以上官员、涉一般官员负面、恶搞央媒、涉政不规范/负面表述、错误配图
军队与安全：抹黑军队形象、歪曲军事行动、涉军个人负面、挑拨军政军民关系
维稳与管制：攻击网络管制、负面炒作安保维稳场景、个人/群体过激维权、拉横幅/大字报、涉政维权行煽
涉外与媒体：境外/反动媒体导向有误、引用外媒错误导向、涉港澳台及国际政治导向有误
舆论与事件：集纳炒作涉政/社会负面、歪曲敏感事件、利用涉政元素进行娱乐化营销

**注意事项**：
导向优先：涉及核心党政军及体制导向问题，按最严标准处理
主体判定：标题、正文、封面、缩略图任一出现，且事件为核心论述或篇幅占比≥1/3即为命中
区分级别：不同级别官员、不同来源媒体、不同涉外主体需按细则区分
豁免情形：历史文献、合理影视化创作、持批判态度的客观讨论、正规新闻转引，均按规则豁免
合规传播：对外媒利我观点应持支持态度，对非底线负面应批判回应，不得借其抨击中国体制或社会整体

### B. 领导人
**定义**：涉及现任/历任党和国家领导人及其直系亲属、周边元素（姓名、肖像、职务、语录等）的所有内容。重点防范：形象与表述负面、虚假与泄密、借势营销与“低红高黑”、亲属不当关联等风险。

**领导人范畴**：
N1&PF：习近平、彭丽媛。
6C+H8：李强、赵乐际、王沪宁、蔡奇、丁薛祥、李希、韩正、李书磊。
现历任正/副国级、现历任政治局常委
历任/已退健在领导人、特定敏感政治局委员。
亲属/姻亲：直系亲属（配偶、父母、子女等）及关联姻亲、人际关系。

**风险场景**：
1. 形象类：丑化/恶搞/变形、张冠李戴、涂抹打码、冒充、黑白照（非常规）、场景/动作负面嫁接；非官方漫画；无关文强插领导人图片；“批斗照片”。
2. 表述类：攻击谩骂、诽谤、隐晦影射，调侃口误/玩梗，封建称呼，恶意捧踩；排版与称谓不规范（错别字、缩写、单独飘色等）、信息/排序错误；常见语录的负面改造或错引；表情/颜文字导致不严肃画风；无关文章下提及。
3. 虚假/泄密：编造经历与语录、派系斗争臆测、职务/行程未发布前预测、私生活披露、行程偷拍等。
4. 间接风险：借领导人营销/诱导互动；低级红/高级黑、绝对化修辞（“唯一/只有/真正的”）、“亲自体”；借领导人维权；讨论周边细节（品牌、用品、办公室布置等）。
5. 亲属相关：直系亲属负面（攻击/诽谤/封建称呼/影射/捧踩）、现历任正副国级亲属主体或动态盘点、政治局常委姻亲/财产盘点；不规范称谓与人际关系牵连、无关文下强行关联。
6. 专项：赵紫阳主体/缅怀；已退/敏感政治局委员回顾/怀念/歌颂；李强同名引发误导。

**注意事项**：
取最严原则：涉及体制影射或 N1/核心领导人，优先按更严条目处理。
主体判定：标题提及/内容核心聚焦/相关篇幅 > 1/3 即视为命中主体；区分“三家/规范稿源/自媒体”。
豁免示例（需无负面导向）：历史原因的黑白老照片/报纸小图；普通字幕遮挡；泰森身上毛泽东纹身、印有毛泽东的服装/帽饰；毛、周、邓等历史简称/尊称；王毅新旧职务称谓并存；相关文章下对胡/温/李克强一般性缅怀或客观描述；中性引用 N1 常见语录；无关文下引用王毅表述。
评论区规则：领导人主体文下仅放强正面、纯正向；中性/无关/低质/不严肃、表情颜文字、引流、对比拉踩等从严。
漫画/图片：领导人漫画仅限官方公开；出现批斗照或非官方漫画即命中。
名词与表达：禁止封建称呼、拼音/缩写、错别字、职务排序错误；“亲自体”“唯一/真正/只有”等绝对化语式从严。

### C. 指令舆情
**定义**：针对平台接收到的监管、机构或平台内部临时/长期专项治理指令，涉及涉政、涉外、涉军、涉敏舆情等特定领域内容的识别与处置，确保内容导向符合政策与舆论管控要求。

**指令舆情范畴**：
短期舆情：反日仇日、试题泄密、叙利亚局势关联中国投资、涉政标题封面夸大、邵氏电影超长片段、胡耀邦忌日相关、举牌违规等
长期专项：俄乌战争敏感/风险内容、兜底类问题（无翻译方言、领导人著作偏好、疫情发布会音频不当使用、涉时政配图等）、长城保护、炒作“牛马”负面、展示枪支性能、“毒攻略”违规出游、巴以冲突歪曲表述、美化/鼓吹特朗普及其团队或亲属等
ZL类指令：小端、压后台、详情、自见、删除、重点文章、全端禁转发等执行要求
其他涉政涉敏类：中东冲突夸大恐慌、宣扬特定政治人物政策、娱乐化造梗等

**风险场景**：
1. 涉及国家、地区敏感冲突（俄乌、巴以、中东等）中存在片面支持、贬低、丑化、夸大、鼓吹战争等导向问题
2. 涉及中国政治人物纪念、领导人相关书籍喜好、外交敏感、涉军涉密等内容
3. 涉及特殊敏感事件或话题的夸大化标题封面、虚假或引导性叙事
4. 组织、宣传或引导参与违规活动（攀爬野长城、非法/危险出游路线、“毒攻略”等）
5. 炒作消极社会观念（“牛马三件套/四件套”）、展示枪支性能
6. 特定文化娱乐内容（如邵氏电影超长片段、举牌行为）触发专项管控
7. 命中平台ZL指令类处置要求（可见度限制、互动限制、删除等）

**注意事项**：
1. 严格区分导向：批判性、客观报道、警示类内容在不触发指令的前提下可正常审核放出
2. 判断是否满足标签条件需结合标题、封面、正文主旨及篇幅占比
3. 涉外、涉政、涉敏话题需重点关注与中国的关联度及导向倾向
4. 特殊账号（如国新媒体、驻外使领馆账号）按对应豁免或专门规则执行
5. ZL标签内容需严格按指令执行，包括可见度限制、互动限制及转发权限控制

### D. 敏感人物/组织/作品
**定义**：涉及极高危、高危、中危、中低危、低危敏感人物与组织，以及违禁作品、分裂势力、暴恐势力、邪教组织及相关内容的风险域。重点防范传播、宣扬、支持、褒扬此类主体或其相关有害信息，防止导向错误、虚假信息、违法内容及国家安全风险。

**敏感人物/组织/作品范畴**：
敏感人物与组织：涵盖极高危、高危、中危、中低危、低危人物与组织（包括劣迹艺人、邪教人物等），及相关事件。
违禁作品：极高危、高危、中危、低危违禁影视、文艺、音视频作品及衍生内容。
分裂相关：分裂组织、分裂标识、分裂言论、领土错误表述、地图缺失、涉台/涉港澳等导向问题。
暴恐相关：暴恐组织、人物、场景、信息画面、极端宗教主义服饰等。
邪教相关：法轮功及其他邪教内容与人物。

**风险场景**：
直接呈现：标题、正文、封面、缩略图中出现上述人物、组织、作品、标识、场景等。
导向错误：对敏感主体持支持、美化、洗白、同情、褒扬等态度；与官方口径不一致；借其关联党政有害内容。
违规内容：传播买卖违禁作品资源；传播分裂、暴恐、邪教教义思想；错误地图、分裂标识未打码；宣扬暴恐组织或行为。
间接风险：评论区导向错误；无关场景下的不当引用或展示；利用相关主体进行不当营销或炒作。

**注意事项**：
取最严原则：同一内容命中多条规则时，按最严标准处理。
主体判定：标题提及、内容核心论述、篇幅占比≥1/3、封面出现等均视为命中。
区分态度：批判、科普、客观描述可按规则豁免，褒扬或模糊导向从严管控。
区分危害等级：极高危不分稿源管控；高危及以下视情况区分稿源。
特殊豁免：规范来源的批判性报道、官方口径一致的客观介绍、历史回顾中不含导向错误的内容等可放行。

### E. 违法违规
**任务与边界**
目标：高召回识别一切触达“违法违规”一级风险域的内容，尤其是含推广/引流/教学/交易/非批判性展示/承诺收益/技术规避要素的场景。
判定基准：出现以下任一即命中：
a) 直接违法行为的展示/教学/交易/招募/中介；
b) 联系方式/交易渠道/二维码/外链/引导私信等引流；
c) 承诺或暗示收益并结合标的/服务/操作建议；
d) 规避监管/制度漏洞/技术破解/危险物品制作与使用。
证据优先级：标题 > 封面（含水印/文案）> 正文 > 评论置顶/精选 > 外链锚文本/二维码说明 > 话术截图。
豁免仅在“主旨明确为批判/科普/权威通报”且无引流/联系方式/教程/交易时成立；若二者并存，以命中为准。

**违法违规定义**：“违法违规”指违反国家法律法规、行政规章或公序良俗的内容或行为，包含但不限于：
金融与经济：财经操作建议、财经类诱导、非法贷款/外汇、资金跑分、征信修复/非法征信、债务优化/代办退保、彩票违规等；
公共安全与社会秩序：涉毒、涉枪涉爆、危险物品/管制刀具、危害公共安全物品/行为、赌博、诈骗、假证、违法通信、暗网等；
信息与隐私：泄露/买卖/教授获取公民个人隐私信息、获客类软件、电销卡等；
生态与文保：长江禁渔/非法捕鱼、国家保护动植物、盗墓等；
违法产品/服务：主板机、水军、公关删帖、枪手/代写、游戏私服/外挂、破解/作弊软件、非正规加油车、疑似假货、烟草/槟榔、小产权/军产房、特供/专供品、医疗/药品/健康类违规推广等；
意识形态及AI风险：AI生成涉军/涉政/重大会议/社会负面新闻/红墙青年/名人仿冒广告等；
政策/制度漏洞与技术规避：高考移民/证件挂靠/特殊景区违规、翻墙/绕开实名/物联网卡滥用、无人机违规改装/禁飞违飞等；
涉华非法移民/三非/境外打工移民夸大宣传等。

**高危触发要素（任一满足即召回）**：
推广/交易/引流：出现价格、购买方式、渠道、邀请码、代理、私信/微信/QQ/电报/WhatsApp/小红书/公众号/小程序、群号、二维码、主页点我、评论区留号、“加V”“vx”“威信”等规避写法。
操作建议/教程/经验分享：教学、流程、脚本、秘籍、线路、避查技巧、一对一指导、“拿代码/拿名单/下工具”。
承诺/暗示收益：稳赚、翻倍、躺赚、黑马/牛股、日/周/月入X、保本、跟单/合买、诊股/荐股、信号/指标/内参。
技术规避/违法工具：破解/外挂/刷量/社工库/定位/开锁/物联网卡/暗网/翻墙/代理池/改机/改号/主板机。
危险物品/公共安全：枪支弹药/爆炸物/管制刀具/危险化学品/翻牌器、脱轨器、特供酒、军警用品真实制式等。
生态与文保：长江禁渔、电鱼/炸鱼、国家保护动植物买卖、盗墓实操。
AI敏感：AI生成我国军警/重大会议/涉政负面/社会负面新闻/红墙青年、名人仿冒广告、AI接吻教程/程序、AI复活逝者名人等。
涉移民“三非”/走线：走线攻略/中介/队友招募、伪造迫害证据/话术、境外打工夸大福利。
医疗/药品/健康黑灰产：违规广告导流、代孕、鉴别胎儿性别+联系方式、长寿药/壮阳/增高等承诺疗效。

**范畴→细则 映射与检索模板（召回清单）**：
1 金融与经济
财经操作建议/财经类诱导：
触发词：荐/诊/问股、黑马/妖股、翻倍、看涨/利多、K线/指标/内参/公式/源码、VIP群、实盘带单、一对一教学。
规则：如出现承诺/暗示收益 +（标的或服务/联系方式/引流）→ 命中。

非法贷款/代办退保/非法兑换外汇/征信修复/非法征信/债务优化/资金跑分/彩票违规：
触发词：网贷/美容贷、停息挂账/反催收、花呗/备用金提额、征信洗白/修复/下户、支付宝红包跑分、兑换外汇私下、合买/跟单/私房菜、私信开单。
规则：出现服务宣传/教程/联系/价格任一→ 命中。

2 违规广告 & 违法产品/服务
医疗/药品/健康：
触发词：代孕、鉴别胎儿性别、长寿药/壮阳/增高、三甲医院预约/挂号外联、莆田系/高危专科榜单推广。
规则：一旦出现导流/联系方式/优惠→ 命中；“批判/科普”无引流可豁免。

水军/虚假流量/账号买卖/删帖公关：
触发词：刷量/涨粉/投票/上热搜、删差评、KOL引爆、主板机群控。

游戏私服/外挂、破解/作弊软件：
触发词：一键脚本、透视、自瞄、加速器脚本、免激活、注册机、外挂开源/下载。

非正规加油车、POS机买卖、疑似假货、烟草/槟榔、小产权/军产房、特供/专供品、军警用品：
一律以推广/展示+导流/教程/交易为判定核心。

3 公共安全与危险物品/行为
枪支弹药/爆炸物/管制刀具/危险物品制作与拆解：
触发词：图纸/材料/火帽/雷管/装填、实体展示/零件/仿制、制作/组装教程。

危害公共安全：
触发词：翻牌器、脱轨器、卫星锅/境外接收、开锁教程、暗网访问、无证驾驶等。
规则：出现教学/工具售卖/导流即命中。

4 信息与隐私
泄露/买卖/教授获取公民隐私信息：
触发词：资料库/一手数据/精准投放、快递面单、小区业主、开房记录、人肉教程/社工库、针孔摄像设备/偷拍视频。
规则：非当事人主动曝光 +（传播/买卖/教程/工具）→ 命中。

获客软件/电销卡：
触发词：评论区抓取电话/微信、AI外呼、企业名录、爬虫抓取粉丝等。

5 生态与文保
长江禁渔/非法捕鱼/电鱼炸鱼/国家保护动植物/盗墓：
触发词：通江湖泊/支流清单、电鱼器、野味买卖、象牙/虎皮/熊掌、金属探测器盗掘。
规则：展示/教学/交易/食用宣传→ 命中；科普/反面案例无引流可豁免。

6 AI敏感与意识形态
AI生成涉军/涉政/重大会议/社会负面新闻/红墙青年/名人仿冒/AI接吻教程/AI复活逝者：
触发词：Midjourney/通义/文心/豆包/DALL·E、水印、Pose/换脸/配音、大会名词清单、红墙青年模板语、仿冒名人带货。
规则：只要生成对象触达清单范围且非批判或含引流/下载/使用教程/程序→ 命中。

7 涉华非法移民/三非/境外打工移民夸大
走线/偷渡/政治庇护话术/“三非”/境外打工夸大福利：
触发词：达连隘口、路线/过境窗口/边检规避、中介/队友招募、PS通缉令/假材料、包永居/全家移民。
规则：攻略/教程/中介/夸大承诺/组队/实时分享→ 命中。

8 无人机/特殊景区/违法通信/彩票等
无人机违规（限速解除、禁飞区/空投器改装）
特殊景区违规攻略（315国道U型、梧桐山、罗布泊彭加木碑）

违法通信/垃圾短信回收索赔教程
彩票违规（承诺中奖、合买跟单、私人定制）
规则：出现教程/导流/联系方式/承诺收益即命中。

**典型结构信号（比关键词更强）**：
价格/库存/发货/团购/返利结构；课程表/课时/优惠券/报名；
二维码/短链/跳链与“加我/进群/主页/评论区获取”；
截图类证据：转账记录、收益曲线、聊天记录、“审核通过”通知；
教程结构：步骤/材料/工具/注意事项/避坑/话术模板；
比较/对照引诱：国内外待遇对比、零门槛/包过/保签/无考试。

**灰区裁剪与豁免（必须全部满足）**：
仅在以下同时成立时可“不过”：
主旨为官方通报/权威媒体新闻、反诈/禁毒/反赌宣传、客观科普或批判；
没有导流/联系方式/教程/交易/下载/加入群/工具分享；
没有承诺或暗示收益、没有“个人服务/接单”。
影视剧/历史影像中出现涉毒/涉枪等画面：仅当不是内容主旨或占比< 1/3 且无引流与教程，方可豁免。

### F. 意识形态
**定义**：涉及影响或挑战国家政治制度、主流价值观、民族团结、历史观、宗教政策等意识形态安全的内容，包括传播、宣扬、解读、恶搞、否定或丑化与官方立场、历史定性、民族宗教政策相违背的观点、信息或行为。

**意识形态范畴**：
历史相关：历史虚无主义（宣扬/批判）、歪曲丑化正面典型、损毁侮辱英烈纪念设施、恶搞革命经典及标志、错误解读党史军史、美化侵略行为、传播错误史观、否定爱国主义教育、历史事件敏感/负面、中共历史敏感人物。
民族相关：民族负面有害、民族及地区负面言论、天葬报道、民族地区敏感话题、民族地区维稳政策等。
宗教相关：非法传教、解读宗教经文教义、宗教聚集类展示、宗教负面有害、非法宗教活动、宗教不当关联、宗教影音制品等。

**风险场景**：
1. 歪曲、否定、恶搞、侮辱党史国史、英烈、民族文化、宗教信仰的内容。
2. 宣扬历史虚无主义、历史否定主义、美化侵略者、美化民国或民族分裂势力。
3. 传播民族歧视、仇恨、刻板印象，泄露民族地区敏感信息。
4. 宣扬或诱导信教，教授宗教教义、组织非法宗教活动、商业化利用宗教，恶搞或攻击宗教领袖与信众。
5. 描述、展示敏感历史事件或民族地区维稳政策的细节画面。

**注意事项**：
重点识别是否含有推广、引诱、美化、否定、恶搞等导向倾向。
对批判性、科普性、正面引导性内容可酌情豁免。
涉及历史人物、事件、民族风俗、宗教活动等，应结合上下文及官方定性判断。
对含隐晦变体、谐音、图文结合规避的内容需综合判定导向。

### G. 公司法务
**定义**：涉及泄露公司机密信息、传播公司及其产品/高管负面内容、侵犯版权等，可能引发法律、合规、商业及品牌风险的内容。
 
**公司法务范畴**：
公司相关：涉公司泄密、以公司名义采访/报道、涉公司话题（高收入主播、APP负面、内涵段子事件、高管负面等）、企业负面/黑稿、一鸣首富相关内容。
版权要求：未授权影音/漫画/央视版权、枪版院线电影、NBA版权、版权黑词等。

**风险场景**：
1. 泄露公司内部审核规则、战略调整、推荐机制、机密资料，或未经授权使用公司名义进行报道。
2. 披露或炒作公司及旗下产品、高管的负面消息、错误信息或敏感事件。
3. 传播未授权或侵权的影视、歌曲、漫画、赛事等内容，包括网盘资源、送审样片、枪版院线片、NBA/WNBA比赛画面等。
4. 搬运版权黑词媒体的唯一稿件。

**注意事项**：
区分正常业务宣传、合作报道与违规泄密或冒用公司名义的内容。
判断是否涉及具体收入数据、负面评价、极端情绪或恶意攻击。
版权内容需核实授权情况，注意豁免账号、白名单与非主体转载情况。
涉及敏感话题时，结合上下文判断导向与真实性，优先适用高风险规则。

### H. 不良价值观
**定义**：指宣扬、鼓动、展示违反社会主义核心价值观、违背道德法律、公序良俗，或可能引发社会极端情绪、群体对立、模仿风险的不当言行与价值取向。

**不良价值观范畴**：
饭圈文化：控评、刷量、冲榜、跟拍代拍、集资应援、粉丝互撕拉踩、非理性消费观念等。
公序良俗：违背伦理纲常、炫富拜金、侮辱受害者、虐待动物、歧视攻击群体或地域、宣扬极端思想（献忠学、极端男/女权等）、号召网络暴力、家庭暴力、危险饮食噱头等。
民粹主义：极端民族主义或排外言论、幸灾乐祸、鼓吹战争、道德绑架、反科学态度、恐慌言论、军事威胁论。
商业清朗：恶意诋毁、攻击、造谣国内重点企业或企业家，煽动抵制或传播虚假负面信息。
网络暴力：恶俗辱骂、人格侮辱、造谣抹黑、祖安文化、诅咒用户等。
危险行为：对人造成实际伤害的恶搞、高空极限运动（无防护）、车辆行驶中危险动作。

**风险场景**：
1. 宣扬违法、暴力、歧视、侮辱、仇恨、极端对立等言行。
2. 鼓励非理性、极端或不安全的群体行为。
3. 渲染低俗、恶俗、拜金享乐及不当社会风气。
4. 捏造、恶意揣测、污名化企业/人物，破坏营商与舆论环境。
5. 通过言论或展示内容引导模仿危害健康、安全的行为。

**注意事项**：
关注是否为标题提及、核心论述对象或篇幅占比≥全文1/3，满足即判定。
对批判、科普、新闻报道类内容且导向正确可酌情豁免。
同时命中多条规则时，按最严标准执行。
评论区同样适用，含恶俗辱骂、歧视攻击、号召不当行为需管控。
封面/缩略图内容与正文一致纳入判定范围，涉及不当元素需干预。

### I. 色情低俗
**定义**：
 涉及露骨或隐晦展现性行为、性器官、涉性恶俗、低俗性暗示，以及传播淫秽色情作品、资源或服务等，易引发性联想、性刺激或低俗画风的问题内容，包括真人、非真人（动漫、AI、画作、雕塑等）及文字、图片、音视频等多种形式。

**色情低俗范畴**：
色情底线：性器官/仿真性器具露点透点、性行为画面/音频、性感受描写、类性交行为、性暗示挑逗、淫秽作品、色情价值观、色情服务与资源等。
涉性恶俗：性爱体验、红灯区、特殊性癖、涉性恶搞画面、骆驼趾/激凸、低俗涉性标题等。
轻微低俗：低俗擦边虚构内容、两性话题与性科普、涉性灰黑产与产业话题、成人影片人物、低俗化婚恋观与低俗场所展示、低俗诱导画面等。
LGBT：同性题材文学娱乐、生活纪实、同性恋激情片段、同性恋交友/交易信息、同性及伦理影视等。

**风险场景**：
1. 直接或突出展现性器官、性行为、仿真性器具等露骨内容。
2. 文字、音视频描写或暗示性行为、性感受、性能力，或使用道具、动作模拟性交。
3. 涉性恶俗话题、特殊性癖、涉性恶搞画面及低俗化身材描绘。
4. 传播、宣传、交易淫秽色情作品、色情资源、性服务或性交易信息。
5. 使用低俗涉性标题、封面引流，或在非相关内容中嵌入低俗性暗示元素。
6. 低俗擦边类娱乐、搞笑或科普内容，涉性灰黑产、产业分析、成人影片人物等内容篇幅过多。
7. LGBT题材中过于亲密的画面、涉性暗示、交友/交易信息或特定剧情演绎。

**注意事项**：
优先判定直接露骨、交易、传播类内容，严格管控。
结合内容形式、语境和导向判断是否为恶俗或低俗性暗示。
对历史类、纪录片、新闻报道、时尚走秀、正能量科普等符合豁免条件的内容可放行。
对封面、缩略图命中风险的，按封面干预规则执行。
真人与非真人（动漫、AI、画作、雕塑）均需适用同等标准，逼真程度高的画作按真人管控。

### J. 内容质量
**定义**：指作品在信息完整性、可读性、准确性及合规性上的质量问题，包括内容空洞、画面无信息量、排版混乱、标题误导、功能异常、链接不可用、抽奖活动违规等，影响用户理解与平台内容生态健康。

**内容质量范畴**：
内容低质：无意义内容（纯符号/数字/表情/无信息链接）、画面无信息（黑屏/模糊/图裂）、阅读困难（乱码/排版混乱）、标题党（标题歧义/题文不符/旧闻新发）、功能异常（空数据/缺失信息/双层图）、无关评论。
外文/少数民族语言：涉国内时政或主题不明，主体为外文或少数民族语言且无中文翻译。
内外链问题：引用的内链处于不可访问状态（压后台/自见/删除）或外链打不开/报错。
创作规范—抽奖活动不合规：参与方式、奖品设置、活动信息不符合平台规则。

**风险场景**：
1. 发布纯标点、纯链接、乱码、全黑屏/白屏画面。
2. 标题与内容不符、故意制造歧义、将旧闻冒充新闻。
3. 涉国内时政类内容仅用外文且无中文译文。
4. 链接（内链/外链）无法访问或内容缺失。
5. 抽奖要求违规操作（如点赞+评论参与）、奖品不规范（虚拟或金额异常）、活动时间设置不当。

**注意事项**：
小视频体裁可豁免纯符号标题。
历史类、非涉政外文内容可正常审核。
链接仅需审核首跳页面，不需无限跳转。
抽奖需确认作者资质，且参与方式、奖品、活动信息均合规。

### K. 营销行为
**定义**：指利用平台内容进行商品、服务、平台、活动、筹款等直接或间接推广、引流、售卖的行为，包括明确交易路径、联系方式、二维码、外链等形式，可能影响平台内容生态和用户安全。

**营销行为范畴**：
联系方式/二维码：含个人联系方式（微信、QQ、电话等）或突出展示二维码（≥图片面积1/2）。
硬广告：直接售卖或引导购买商品/服务的链接、联系方式、二维码，推广性网址/口令/邀请码，以及借国际局势引流售卖应急物资。
未授权平台导流：引导至快手、B站、知乎、夸克等未授权平台消费、关注、下载或观看完整版内容；小说搬运引流至未授权平台；诱导私信获取资料；特定“头部同步”来源含外链。
社会求助：展示捐款二维码、账号或其他筹款方式的求助信息（含变相要钱）。
封建迷信：
强封建迷信：宣传、介绍、鼓励参与婴灵超度、冥婚、补财库等迷信活动；
弱封建迷信：风水、算命、相面等命理预测内容；
封建迷信虚构创作：含强封建迷信元素且明确为虚构作品的内容。

**风险场景**：
1. 标题/正文/配图含微信号、电话、QQ群、显著二维码。
2. 提供购买商品/服务的直接链接、扫描二维码下单、输入口令领券等。
3. 明示/暗示引导至未授权平台或通过私信获取资源。
4. 文章含筹款二维码/账号，直接面向公众募捐。
5. 内容渲染封建迷信活动功效、提供实施指引或进行命理推算。

**注意事项**：
多类场景有豁免条件，如非刻意展示、公共服务联系方式、活动/展会海报、版权转载声明、字节系产品导流等。
批判性、科普性、非推广目的的相关内容可正常审核。
封面出现违规元素时，优先用封面图按钮干预。
对虚构作品需有明确“故事/纯属虚构”等声明，否则按强封建迷信处理。

### L. 事件
**定义**：指平台内容中涉及国内外重大政治历史事件及相关群体活动的信息，包括直接提及、隐晦暗示、日期指代、衍生活动、国内民运等，可能引发重大舆情风险或与官方立场相悖的表述。

**事件范畴**：
六四相关：直接或间接提及1989年春夏之交事件，包括错误表述、相关人物/组织/作品、衍生活动（如维园烛光晚会、天安门母亲等）。
隐晦关联六四：以隐喻、暗示、符号（如数字、日期、表情、蜡烛）或突发巧合等形式影射六四，含调侃维稳、炒冷饭等。
六四日期指代：使用“六四”“6·4”等日期形式直接指代事件。
国内民运事件：如太阳花运动、茉莉花革命、全民共振、七一大游行、雨伞革命/占中、反送中、光州事件等。

**风险场景**：
1. 标题、正文、封面、缩略图中出现上述事件的直接描述、暗示性表达或相关符号。
2. 使用与官方基调相悖的词汇指代事件（如“流血”“大屠杀”等）。
3. 借事件炒作、蹭热点，或在无关语境中影射事件。
4. 评论区出现与正文无关但提及上述事件的内容。

**注意事项**：
与事件相关的批判、煽动、引导性内容均需管控，不分稿源。
与事件无关的普通活动（如支联会、维园集会）可在不涉及事件的情况下正常审核。
对于日期指代和衍生活动需结合语境判断是否存在规避或影射意图。
符合官方口径的中性表述可正常通过。

### M. 未成年人
**定义**：未成年人风险域指涉及未满18周岁人群的违法违规、低俗不良、引导性错误内容。仅当未成年人是叙述主体/核心事件的关键参与者，且存在风险行为或不良引导时才命中。
不得仅因出现“学生”“孩子”等字样而命中。

**未成年人范畴**：涵盖所有年龄未满18岁的自然人，包括儿童与青少年，在任何线上线下场景中均适用本规范。

**命中场景**：
A. 违法犯罪类
涉及未成年人实施或遭受违法犯罪（如盗窃、诈骗、暴力、性侵、虐待、拐卖）。
涉及未成年人参与涉毒、涉黄、涉赌等非法活动。
传播涉及未成年人的违法影像、照片、隐私信息。

B. 色情低俗类
描述、展示未成年人裸露或性暗示内容（含绘画、动画、AI生成）。
任何将未成年人性化、物化的表述或画面，即便是虚构情境。

C. 不良价值观类
宣扬未成年人早恋、婚孕、同居等不当关系。
鼓吹未成年人沉迷赌博、暴力、酗酒、吸烟等不良习惯。
教唆、诱导未成年人参与违法或危险行为（如飙车、极限挑战）。

**豁免场景（不命中）**：
正常新闻报道、政策宣传、公益教育（如防拐、防沉迷、防性侵）。
文学、影视、历史内容中未成年人并非核心违规主体或情节健康。
批判、讽刺、揭露未成年人风险行为且具有警示意义。
泛指群体（如“青少年科技创新大赛”）且无违规指向。

**边界与交叉处理**：
核心论述对象原则：如果未成年人只是配角或旁述，不命中。
交叉风险优先级：若涉政、涉军、涉社会重大新闻为主，应归主风险域，未成年人作为次要因素不单独命中。
时间维度判断：涉及未成年人长大后的事件，不按未成年人命中，除非内容明确回溯其未成年时期的风险行为。
虚拟内容处理：即便是二次元、虚拟角色，只要形象、设定、年龄属性为未成年人，且存在风险行为，同样命中。

**特殊提醒**：
需同时关注隐性指代（如“小学生”“初中女孩”）是否构成未成年人身份。
对于图文混合、短视频等，文字和画面任何一方符合风险条件即命中。
模糊案例优先依据内容整体导向与核心风险对象来判定。

### N. 虚假不实
**定义**：指内容存在不可核实或明确虚假的信息，包括未经权威渠道证实的消息、捏造事实的谣言、具有误导性的虚构叙述等，可能造成公众误解或混淆视听。
不可核实或明确虚假：以官方消息及权威网站（政府部门、主流媒体、权威机构）发布为准；若无相关发布，或权威发布与内容描述明显不一致，即为不可核实或明确虚假。

**虚假不实范畴**：
适用于以下类型，不论稿源：
虚假谣言：除涉国内/涉中时政、历史虚无、涉领导人等需走更严规则外的，不可核实或明确虚假的信息。
名人去世虚假：发布未经证实或明确虚假的名人去世消息，或基于虚假死亡信息展开的回顾、缅怀、盘点等。
名人去世标题党：标题刻意隐藏关键信息或辟谣信息，以误导受众博取流量。
疑似谣言：命中疑谣库提示的疑似谣言，且未涉国内/涉中时政、涉领导人等更严范围。

**触发条件（满足任一即判定）**：
标题提及虚假/疑似虚假信息
以虚假/疑似虚假信息为核心论述对象
虚假/疑似虚假信息篇幅 ≥ 全文 1/3

**重要豁免（易误召场景明确排除）**：
以下情形不判定为虚假不实：
辟谣、澄清、批判类内容（需明确包含辟谣意图或事实更正信息）
以第三方视角客观报道有虚假信息的存在（如“网传”“据传”“传闻称”等中性引用），且未将其当作事实传播
历史影视、文学创作、讽刺、比喻等明显虚构场景
仅引用知识库关键词但语境与谣言主题无关
封面图或缩略图命中虚假内容，但正文未涉及且可通过封面图干预方式解决

**边界说明（减少误召关键点）**：
引用但非传播事实：若文章只是分析、评论“有谣言出现”这一现象，而非传播谣言本身，不应命中
疑谣库触发：需综合上下文研判，而非单凭关键词
非涉事实判断的观点类（如感叹、情绪、隐喻）不算虚假
历史背景或文学化叙述需确认是否在传播当前社会的可核实事实，若仅为回顾或引用，不命中

**执行优先级**：
涉国内/涉中时政、涉领导人相关谣言 → 按更严规则处理
其他场景 → 按上述虚假不实标准处理

### O. 引人不适
**定义**：指通过视觉、听觉或内容描写等方式，引发用户强烈不适、恐惧、反感或生理排斥反应的内容，不论来源均需管控。

**引人不适范畴**：
 涵盖中度引人不适（密集恐惧、病理特写、违背主流饮食习惯、不当展示特殊群体、意外死亡动物尸体、屎尿、车祸实况等）与重度引人不适（未打码真实尸体、凶杀现场、残忍宰杀、极端猎奇饮食或行为、未打码血腥暴力等）。

**风险场景**：
1. 中度引人不适
密集恐惧画面（大量爬虫等）、病理检查或人体污物清理特写、大面积皮肤溃烂等病症；
食用引发不适的食材（蝙蝠、活蛆奶酪等）、活剥海鲜皮、惊悚吓人画面；
特殊群体非正常展示（残疾/畸形特写）、畸形动物、打码的人类尸体、动物或人屎尿、未打码车祸实况；
丧葬场景结合诅咒意味、攻击丧葬习俗、丧葬类表情包带有诅咒。
2. 重度引人不适
假吃/催吐/大胃王浪费食物、暴饮暴食、过量饮酒、猎奇就餐或行为（生吃活物、喝尿吃屎等）；
未打码真实尸体、实拍杀人过程、动物喷血宰杀；
博眼球猎奇行为（哗众取宠扰乱秩序等）；
含有明显血腥暴力的医疗画面、历史照片、影视/动漫内容或特效化妆；
极端动物攻击人类等严重威胁生命安全的场景。

**注意事项**：
1. 出现在标题、正文、封面、缩略图均可命中，封面命中时优先使用封面图按钮干预；微头条缩略图/视频封面命中则直接管控。
2. 满足核心论述对象或相关内容篇幅 > 全文 1/3任一条件即需处理。
3. 部分历史老照片可豁免；新闻报道、批判类内容、正常饮食习俗、生吃正常食材可豁免。
4. 判断应结合画面清晰度与感官冲击程度，建议疑罪从有。

### P. 社会新闻
**定义**：指国内涉及重大灾难事故、公共安全事件、社会治安案件、宗教负面事件等，具有较强负面影响、易引发社会恐慌或误导公众价值观的新闻信息及相关画面。

**社会新闻范畴**：
 涵盖重大灾难事故、突发疫情、暴力刑事案件、涉未成年负面新闻、涉宗教负面社会新闻，以及暴力冲突或执法现场画面等。

**风险场景**：
重大灾难事故：死亡人数>3人或受伤人数>10人的自然灾害、交通事故、生产事故等。
突发疫情：对公共卫生安全构成威胁的传染病疫情。
暴力刑事案件：杀人、抢劫、强奸、器官买卖、报复社会、拐卖人口等严重危害社会的案件。
涉未成年负面新闻：未成年人涉及违法违规或违背社会道德伦理的事件（无论施害或受害）。
涉宗教负面社会新闻：拆除、改造、关停宗教场所等易引发宗教矛盾的事件。
暴力冲突与执法画面：实拍打架斗殴、暴力伤人、个人冲突、公检法或城管执法现场等，画面占全文篇幅≥1/3的。

**注意事项**：
1. 区分稿源进行管控；标题、核心论述或篇幅≥全文1/3均需处理。
2. 对内容进行辟谣或以案释法的法律科普可豁免。
3. 涉及娱乐明星或体育运动员的相关社会事件按娱体垂类审核；台湾地区社会负面新闻暂不管控。
4. 2020年及以前的事件（以结案时间为准）可豁免，画面类内容仍按规则管控。
5. 宗教人士个人违背戒律或形象受损的事件可正常发布。

### Q. 健康医疗
**定义**：涉及疾病诊疗、药品药材、偏方药方、医疗方法、健康科普、医美操作等与人体健康相关的内容，包括中西医理论与实践在疾病预防、诊断、治疗、康复等环节的传播与介绍。

**命中条件**：
1. 无资质账号违规发布
无资质账号发布涉及疾病（含症状）的科普、预防、诊断、治疗类内容；
无资质账号发布任何疾病诊疗相关的偏方/秘方/土方（根据相关字眼识别）、药方/处方/方剂/古方，且含有具体剂量；
无资质账号发布有争议的中西医疗法（如热毒理论、拔罐排毒、针灸治百病等）介绍内容；
无资质账号即使是批判偏方/药方/争议疗法，也需命中（批判豁免仅限有资质账号）。

2. 有资质账号违规发布
有资质账号发布疾病诊疗相关的偏方/秘方/土方、含剂量药方、有争议中西医疗法介绍内容时，依然需命中；
有资质账号仅在批判性介绍上述内容时可放出。

3. 疾病类内容
标题或内容涉及疾病/症状的科普、预防、诊断、治疗，均需命中；
包括中医诊治手段（刮痧、拔罐、针灸、艾灸、穴位按摩等）的具体操作；
中医经络、脏腑理论，只要与具体疾病/症状诊断、治疗、预防直接关联，也需命中。

4. 药品/药材类内容
标题或内容主旨为药品药效、用药指导、药理知识、药方推荐，均需命中；
科普中药材并介绍其治病功效的，也需命中。

5. 医美类内容
教授如何自行实施具有创伤性或侵入性的医美项目（如自行注射玻尿酸、线雕、切割类手术等）需命中；
仅科普介绍医美项目可放出。

**豁免条件**
以下内容可放出，不命中：
药膳、药酒、代茶饮、足浴包类养生保健内容；
孕产育儿、健康生活习惯、生理学科普、医学历史考古类；
无具体配方剂量的医学研究成果；
有资质账号批判性介绍偏方/药方/争议疗法的内容。

**判断逻辑优先级**
1. 先看账号资质（有资质/无资质）；
2. 再看内容类别（疾病/药品/药方/医美等）；
3. 最后应用豁免条件（仅当明确满足豁免标准时放出）。

## 任务流程
1. 仔细阅读需要分类的内容，按顺序从头到尾对照以上17个风险域的关键标准，判断内容是否疑似命中任一风险域。
2. 请务必按顺序执行判别，只有完全不符合任何风险域标准，才能判定为无风险。
2. 谨慎执行每一步，每一步结束之后都自查。
3. 确保输出的分类结果符合分类标准和注意事项。若不符合，则返回第一步重新分析。

## 注意
1. 如果一个案例同时命中多个风险域，请输出最相关的风险域标签大写字母
2. 输出的结果为一级风险域标签，即党政军、领导人、指令與情、敏感人物/组织/作品、违法违规、意识形态、公司法务、不良价值观、色情低俗、内容质量、营销行为、事件、未成年人、虚假不实、引人不适、社会新闻、健康医疗其中一个标签或无风险，除了这18个标签外不可输出其它标签。
3. 对于边界模糊的内容，应优先考虑风险防控，进行风险分类
4. 如内容经过极其严格的判断后确实不属于任何风险域，才可标为"无风险"，对于新出现的风险类型，应根据最相近的风险域进行分类
5. 如果判定为无风险，请输出英文大写字母'R'

现在请结合角色要求、一级风险域关键标准以及你对于内容安全的认知，对用户提供的内容进行判别。
请你在这18个选项里做选择判断，请只输出下列选项其中一个英文大写字母
A. 党政军
B. 领导人
C. 指令與情
D. 敏感人物/组织/作品
E. 违法违规
F. 意识形态
G. 公司法务
H. 不良价值观
I. 色情低俗
J. 内容质量
K. 营销行为
L. 事件
M. 未成年人
N. 虚假不实
O. 引人不适
P. 社会新闻
Q. 健康医疗
R. 无风险
"""

user_prompt = """标题: 武汉大学校长，成了关键角色。武大图书馆事件的真相，可能永远都不会有人知道了，可既然法院判了，就等于恢复了男孩的清白，“奸出妇人口”这种封建糟粕规则，更不能横行于现代大学校园，早该扔进历史的故纸堆里去了。所以，该道歉的道歉，产生影响的赶快消除影响，武汉大学堂堂的985高等学府，也不意味着就不犯错误，知\n 正文: 武汉大学校长，成了关键角色。武大图书馆事件的真相，可能永远都不会有人知道了，可既然法院判了，就等于恢复了男孩的清白，“奸出妇人口”这种封建糟粕规则，更不能横行于现代大学校园，早该扔进历史的故纸堆里去了。所以，该道歉的道歉，产生影响的赶快消除影响，武汉大学堂堂的985高等学府，也不意味着就不犯错误，知错能改善莫大焉，不要毁了一个孩子的一生，更不要让社会上的成年人都谈之色变。\n ocrMerge: 公家设立学堂,是为 天下储人才,非为诸生谋 进取;诸生等来堂肄业, 是为国家图富强,非为一 己利身家。 一—摘自1901年《山东大学堂章程 凤凰网 I 62.5万 阅读 凤凰纲 FENG.C 置顶25-7-3122:46已编辑 【#武大校长回应是否撤销肖同学处分# :还要等上级的安排】#武大校长回应图 书馆性骚扰事件#2025年7月31日晚间, 针对备受关注的“武汉大学图书馆男生涉嫌 性骚扰指控被法院驳回”一事,武汉大学校 长张平文回应了记者的电话求询。武汉大 学是否会撤回或调整对于此前被指控性骚 扰的肖同学的处分?校长张平文表示,学 校正在处理中。至于何时会公布处理结果 的问题,他称还要等上级的安排。(经济 观察报) 武汉大学校长张平文:正在处 理“学生性骚扰指... 23星期二201022 @子午侠 飞象网项立刚 I +关注 25-8-106:05微博原创视频博主 发布于北京 武汉大学这事基本是翻转了,校长张平文的回答 基本上是隐含了要撤销处分,这个什么时候要 做,请上级安排,也有可理解的成分。这事已经 变成了一个全国性的重大與情,不仅对武汉大学 造成了重大伤害,而且对武汉和教育界都有影 响,所以不仅是有撤销处分的,对学术不端也要 有个回答。 我一直说中国人骨子里是善良的,一个人把事做 绝,一定会得到回报。如果不是这位杨同学一直 追杀,一个19岁的年轻学生,一生都会带着一 个阴影。杨同学的追杀终于犯了众怒。\n 健康资质: 无健康资质\n 金融资质: 无财经资质\n 账号类型: 自媒体\n 是否规范信源: 否\n 是否为三家: 非"""

completion = client.chat.completions.create(

    model="ep-20250815113624-drl4t",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],

    extra_headers={'x-is-encrypted': 'true'},
)
print(completion.choices[0].message.content)

/Users/bytedance/PythonProgram/test/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


A


#### 数据处理

In [ ]:
### 去掉gid的引号 ###
import pandas as pd
import csv

df = pd.read_csv("data/2w5训练集.csv", encoding="utf-8")
# 去掉多余的引号，只保留数字
df["gid"] = df["gid"].astype(str).str.replace('"', '', regex=False)
# 保存
df.to_csv("data/2w5训练集.csv", encoding="utf-8-sig", index=False, quoting=csv.QUOTE_ALL)

In [ ]:
### 训练数据分布，统计各一级风险域数量和比例，带abcd mapping ###
import pandas as pd

# 定义映射表
mapping = {
    "A": "党政军",
    "B": "领导人",
    "C": "指令與情",
    "D": "敏感人物/组织/作品",
    "E": "违法违规",
    "F": "意识形态",
    "G": "公司法务",
    "H": "不良价值观",
    "I": "色情低俗",
    "J": "内容质量",
    "K": "营销行为",
    "L": "事件",
    "M": "未成年人",
    "N": "虚假不实",
    "O": "引人不适",
    "P": "社会新闻",
    "Q": "健康医疗",
    "R": "无风险"
}
df = pd.read_csv("data/A头条2w训练集.csv")

# 数据总数
total = df.shape[0]

# 同时统计数量和比例
counts = df["一级风险域"].value_counts()
ratios = df["一级风险域"].value_counts(normalize=True)

# 合并为 DataFrame
result = pd.DataFrame({
    "数量": counts,
    "比例": (ratios * 100).round(2).astype(str) + "%"
}).reset_index()

# 调整列名
result.columns = ["代号", "数量", "比例"]

# 添加对应风险域
result["风险域"] = result["代号"].map(mapping)

# 调整列顺序
result = result[["代号", "风险域", "数量", "比例"]]

# 打印对齐的表格
print(result.to_string(index=False))
print("\n数据总数:", total)

代号        风险域    数量     比例
 R        无风险 14899 52.77%
 A        党政军  2516  8.91%
 B        领导人  2237  7.92%
 C       指令與情  1203  4.26%
 F       意识形态   921  3.26%
 H      不良价值观   918  3.25%
 K       营销行为   768  2.72%
 N       虚假不实   536   1.9%
 J       内容质量   534  1.89%
 E       违法违规   524  1.86%
 O       引人不适   503  1.78%
 D 敏感人物/组织/作品   449  1.59%
 G       公司法务   436  1.54%
 P       社会新闻   416  1.47%
 I       色情低俗   407  1.44%
 Q       健康医疗   405  1.43%
 M       未成年人   383  1.36%
 L         事件   181  0.64%

数据总数: 28236


In [ ]:
import pandas as pd
import csv

# 只读入一次，保持字符串类型
df = pd.read_csv("data/2w5训练集.csv", dtype=str)

target_cols = [
    "抽检审出日期","体裁","gid","title","content","ocrMerge",
    "healthComputedName","financeComputedName","mediaType",
    "normativeInformationSource","informationPlatformMark",
    "一级风险域","三级policy"
]

def strip_outer_quotes_if_safe(s: str) -> str:
    if not isinstance(s, str):
        return s
    # 仅当去掉外层引号后仍是“安全字段”（不含逗号/换行）才去掉
    if len(s) >= 2 and s.startswith('"') and s.endswith('"'):
        inner = s[1:-1]
        if (',' not in inner) and ('\n' not in inner) and ('\r' not in inner):
            return inner
    return s  # 其它情况保持不动（以维持 CSV 合法性）

for col in target_cols:
    if col in df.columns:
        df[col] = df[col].apply(strip_outer_quotes_if_safe)

# 写出：最小必要引用，不会给所有字段再套引号
df.to_csv("data/output.csv", index=False, quoting=csv.QUOTE_MINIMAL)

In [ ]:
### 检查两个文件 gid 重叠，如有保留在df1的，删除在df2的

import pandas as pd

# 读入两个 csv
df1 = pd.read_csv("data/营销行为（一线）.csv")  
df2 = pd.read_csv("data/营销行为（质检）.csv")

# 取交集
common_gids = set(df1["gid"]) & set(df2["gid"])

print(f"两个文件有 {len(common_gids)} 个重叠的 gid")

# 保留 df1（不变）
df1_keep = df1.copy()

# 在 df2 里删除这些 gid
df2_new = df2[~df2["gid"].isin(common_gids)].copy()

print(f"删除后 df2 还剩 {len(df2_new)} 行")

# 可选：保存回文件
df1_keep.to_csv("data/test.csv", index=False, encoding="utf-8-sig")
df2_new.to_csv("data/black.csv", index=False, encoding="utf-8-sig")

两个文件有 0 个重叠的 gid


In [5]:
### 合并title, content, ocr ###
import pandas as pd

INPUT = "data/剩余9风险域.csv"
OUTPUT = "data/剩余9风险域.csv"

USER_TEMPLATE = [
    "标题: {title}",
    "正文: {content}",
    "ocrMerge: {ocrMerge}"
]

# 1) 读取并清理表头
df = pd.read_csv(INPUT, encoding="utf-8")
print(f"原行数: {len(df)}")
df.columns = df.columns.str.replace("\ufeff", "", regex=False).str.strip()

# 2) 按 gid 去重
df = df.drop_duplicates(subset=["gid"], keep="first")
print(f"去重后行数: {len(df)}")

# 3) 过滤：一级风险域 == 'R'
# df_a = df[df["一级风险域"].astype(str).str.strip().eq("R")].copy()

# 4) 定义拼接函数
def format_row(row):
    parts = []
    for line in USER_TEMPLATE:
        parts.append(line.format(
            title=row.get("title", ""),
            content=row.get("content", ""),
            ocrMerge=row.get("ocrMerge", ""))
        )
    return "\n ".join(str(p) for p in parts if pd.notna(p))

# 5) 生成 merged_text
df["merged_text"] = df.apply(format_row, axis=1)

# 6) 保存
out_cols = ["抽检审出日期", "gid", "merged_text", "三级policy", "一级风险域"]
df[out_cols].to_csv(OUTPUT, index=False, encoding="utf-8-sig")

print(f"总行数: {len(df)}")
print(f"已导出: {OUTPUT}")

原行数: 16589
去重后行数: 16005
总行数: 16005
已导出: data/剩余9风险域.csv


In [11]:
### 删除空列 ###
import pandas as pd

# 读入 csv
df = pd.read_csv("data/党政军质检.csv", encoding="utf-8")

# 删除掉所有空列（名字为 NaN 或空字符串）
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]   # 先去掉没有列名的
df = df.dropna(axis=1, how="all")                      # 再去掉全是 NaN 的列

# 如果你只想保留特定的几列，可以这样：
keep_cols = ["抽检审出日期", "gid", "title", "content", "ocrMerge", "三级policy"]
df = df[keep_cols]

# 保存结果
df.to_csv("data/党政军质检.csv", index=False, encoding="utf-8-sig")
print("✅ 已保存到 data/营销行为（质检）.csv")

✅ 已保存到 data/营销行为（质检）.csv


In [ ]:
### 重新格式化日期 ###
import pandas as pd

# 读取 CSV
df = pd.read_csv("data/剩余风险域.csv")

# 把 '抽检审出日期' 转换为 datetime
df['抽检审出日期'] = pd.to_datetime(df['抽检审出日期'], errors="coerce")

# 重新格式化为 y/m/d (不保留前导0)
df['抽检审出日期'] = df['抽检审出日期'].dt.strftime("%Y/%-m/%-d")

# 保存回原文件（或者另存为）
df.to_csv("data/剩余风险域.csv", index=False, encoding="utf-8-sig")

In [4]:
### 删除空行 ###
import pandas as pd

# 读入数据
df = pd.read_csv("data/剩余风险域.csv", encoding="utf-8")

# 检查哪些行的 title, content, ocrMerge 都为空
mask = df[["title", "content", "ocrMerge"]].isna().all(axis=1) | (df[["title", "content", "ocrMerge"]] == "").all(axis=1)

# 统计数量
num_empty = mask.sum()
print(f"共有 {num_empty} 行 title, content, ocrMerge 都为空")

# 删除这些行
df_clean = df[~mask]

# 保存结果
df_clean.to_csv("data/剩余风险域.csv.csv", index=False, encoding="utf-8-sig")
print("✅ 已删除并保存")

共有 0 行 title, content, ocrMerge 都为空
✅ 已删除并保存


In [13]:
### 加一级风险域列 ###
import pandas as pd

# 读入数据
df = pd.read_csv("data/色情低俗质检.csv", encoding="utf-8")

# 增加一列“一级风险域”，所有行都填“营销”
df["一级风险域"] = "色情低俗"

# 保存结果
df.to_csv("data/色情低俗质检.csv", index=False, encoding="utf-8-sig")
print("✅ 已增加列 一级风险域，内容为 ''")

✅ 已增加列 一级风险域，内容为 ''


In [ ]:
### gid 去重 ###
import pandas as pd

# 读入两个文件
dfA = pd.read_csv("data/无风险2w.csv", encoding="utf-8")
dfB = pd.read_csv("data/无风险（质检）.csv", encoding="utf-8")

# 找出 dfA.gid 不在 dfB.gid 的行
diff = dfA[~dfA["gid"].isin(dfB["gid"])]

print(f"共有 {len(diff)} 行在 A 里但不在 B 里")
diff.to_csv("data/a_not_in_b.csv", index=False, encoding="utf-8-sig")
print("✅ 已保存到 data/a_not_in_b.csv")

共有 5069 行在 A 里但不在 B 里
✅ 已保存到 data/a_not_in_b.csv


In [12]:
### 合并多个csv ###
import pandas as pd

# 定义要合并的 CSV 文件列表
files = [
    "data/营销行为（质检）.csv",
    "data/共16风险域.csv"
]

for f in files:
    df = pd.read_csv(f, encoding="utf-8-sig")
    print(f"{f}: {len(df)} 行")

# 你期望的标准表头（按你的描述）
EXPECTED = ["抽检审出日期", "gid", "merged_text", "三级policy", "一级风险域"]

def read_clean(path):
    # 统一读取设置：处理BOM、全部按字符串读入，去除列名首尾空白
    df = pd.read_csv(path, encoding="utf-8-sig", dtype=str)
    df.columns = [c.replace("\ufeff","").strip() for c in df.columns]
    return df

# 1) 逐个检查哪些文件含有异常列
has_extra = {}
for f in files:
    df = read_clean(f)
    extra_cols = [c for c in df.columns if c not in EXPECTED]
    if extra_cols:
        has_extra[f] = extra_cols

if has_extra:
    print("⚠️ 以下文件含有期望之外的列（导致合并后多出列）：")
    for f, extras in has_extra.items():
        print(f" - {f}: {extras}")
else:
    print("✅ 所有文件列名都与预期一致，没有多余列。")

# 2) 强制对齐列：只保留 EXPECTED 里的列，缺的列补空
aligned = []
for f in files:
    df = read_clean(f)
    for col in EXPECTED:
        if col not in df.columns:
            df[col] = pd.NA
    # 只保留标准列并按顺序排列
    df = df[EXPECTED]
    aligned.append(df)

# 3) 合并 & 保存
df_merged = pd.concat(aligned, ignore_index=True)
out = "data/共17风险域.csv"
df_merged.to_csv(out, index=False, encoding="utf-8-sig")
print(f"✅ 合并完成，共 {len(df_merged)} 行，已保存到：{out}")


data/营销行为（质检）.csv: 2716 行
data/共16风险域.csv: 34480 行
✅ 所有文件列名都与预期一致，没有多余列。
✅ 合并完成，共 37196 行，已保存到：data/共17风险域.csv


In [16]:
### 统计各标签数量和比例 ###
import pandas as pd

# 读取数据
df = pd.read_csv("data/实验1:test黑_recalled.csv", encoding="utf-8")

# 确保列存在
if "一级风险域" not in df.columns:
    raise KeyError("❌ CSV 文件缺少 '一级风险域' 列")

# 统计数量
counts = df["一级风险域"].value_counts(dropna=False)

# 统计比例
ratios = df["一级风险域"].value_counts(normalize=True, dropna=False) * 100

# 合并为一个表
summary = pd.DataFrame({
    "数量": counts,
    "比例": ratios.map(lambda x: f"{x:.2f}%")
}).reset_index().rename(columns={"index": "标签"})

# 打印结果
print(summary)
print(f"总行数: {len(df)}")

         一级风险域   数量      比例
0          党政军  821  22.12%
1          领导人  368   9.92%
2         营销行为  286   7.71%
3         违法违规  260   7.01%
4         内容质量  227   6.12%
5         指令舆情  214   5.77%
6         健康医疗  206   5.55%
7         公司法务  203   5.47%
8         虚假不实  201   5.42%
9         意识形态  186   5.01%
10       不良价值观  186   5.01%
11  敏感人物/组织/作品  136   3.66%
12        引人不适   99   2.67%
13        色情低俗   87   2.34%
14        未成年人   80   2.16%
15        社会新闻   78   2.10%
16          事件   73   1.97%
总行数: 3711


In [ ]:
### 划分训练集和测试集 ###
import pandas as pd

# 读取数据
df = pd.read_csv("data/共17风险域.csv", encoding="utf-8")

# 打乱顺序
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 划分 90% 训练集 & 10% 测试集
train_df = df_shuffled.sample(frac=0.9, random_state=42)
test_df = df_shuffled.drop(train_df.index)

# 保存到文件
train_df.to_csv("data/实验1:黑数据.csv", index=False, encoding="utf-8-sig")
test_df.to_csv("data/实验1:test黑.csv", index=False, encoding="utf-8-sig")

print(f"黑数据大小: {len(train_df)}")
print(f"test黑大小: {len(test_df)}")

黑数据大小: 33398
test黑大小: 3711


In [ ]:
import pandas as pd

# 读取 CSV
df = pd.read_csv("data/实验1:test黑_recalled.csv", encoding="utf-8")

# 去重处理
dedup_labels = []
dedup_counts = []

for labels in df["recall_first_labels"].fillna(""):
    # 按逗号分割，去掉多余空格
    parts = [p.strip() for p in str(labels).split(",") if p.strip()]
    # 去重（保持顺序）
    seen = []
    for p in parts:
        if p not in seen:
            seen.append(p)
    dedup_labels.append(", ".join(seen))
    dedup_counts.append(len(seen))

# 新增两列
df["recall_first_labels_dedup"] = dedup_labels
df["num_recall_first_labels_dedup"] = dedup_counts

# 保存
df.to_csv("data/实验1:test黑_recalled.csv", index=False, encoding="utf-8-sig")

print("✅ 已完成，新增列：recall_first_labels_dedup, num_recall_first_labels_dedup")

✅ 已完成，新增列：recall_first_labels_dedup, num_recall_first_labels_dedup


In [17]:
import pandas as pd

# 读取数据
df = pd.read_csv("data/实验1:test黑_recalled.csv", encoding="utf-8")

# 命中判定：一级风险域是否在 recall_first_labels_top5 里
df["hit"] = df.apply(
    lambda row: 1 if pd.notna(row["一级风险域"]) 
                 and pd.notna(row["recall_first_labels_top3"]) 
                 and row["一级风险域"].strip() in [x.strip() for x in str(row["recall_first_labels_top3"]).split(",")] 
                 else 0,
    axis=1
)

# 整体统计
total = len(df)
hits = df["hit"].sum()
accuracy = hits / total if total > 0 else 0
overall_df = pd.DataFrame({
    "总样本数": [total],
    "命中数": [hits],
    "整体准确率": [round(accuracy, 4)]
})

# 每个一级风险域的统计
grouped = df.groupby("一级风险域")["hit"].agg(["count", "sum"])
grouped["准确率"] = grouped["sum"] / grouped["count"]
grouped = grouped.reset_index().rename(columns={"count": "总数", "sum": "命中数"})

# 👉 直接输出 DataFrame（方便复制）
print("—— 裁剪top5一级风险域统计 ——")
overall_df

# print("\n—— 各一级风险域标签准确率 ——")
# grouped

—— 裁剪top5一级风险域统计 ——


,总样本数,命中数,整体准确率
0,3711,2997,0.8076


In [ ]:
### 取某列等于xxx的数据 ###
import pandas as pd

# 读取 csv
df = pd.read_csv("data/实验1:黑数据.csv", encoding="utf-8")

# 筛选出一级风险域 == "营销行为" 的数据
filtered = df[df["一级风险域"] != "营销行为"]

# 输出前几行检查
print(filtered.head())

# 如果需要保存为新的csv
filtered.to_csv("data/neg.csv", index=False, encoding="utf-8")

      抽检审出日期               gid  \
0   2025/8/7  1839719098152972   
1  2025/6/12  1833150344740040   
2  2025/7/30  1838937288220684   
3   2025/8/6  1839715880491015   
4  2025/7/31  1838943870438412   

                                         merged_text       三级policy  一级风险域  
0  标题: 孩子这是热的起热疹了。可以用金银花水洗一洗。重要的是一定不能再让孩子受热了！\n 正...  无资质-健康科普/偏方药方   健康医疗  
1  标题: 各地的相亲会都是女多男少。这不是偶然，是必然。 第一，女的离不开男的，她们以为男的离...         违背公序良俗  不良价值观  
2  标题: 每次在脑子里想好下一场签售要做的事之后一打开xhs就会看到同担已经做过了我真的要闹了...           内容低质   内容质量  
3  标题: 网络上流传着这样的传言，这是跳楼的周口邵医生的车！看看这被红漆写满的侮辱性语言！那些...           虚假谣言   虚假不实  
4  标题: 1991年，一名服刑了40年的犯人刚出狱，便立刻写信给国家副主席，信中只有一句话：“...    历史虚无主义-宣扬传播   意识形态  


In [18]:
import pandas as pd

df = pd.read_csv("flow_data/Ai设定风险黑白.csv", encoding="utf-8")
len(df)

1763

In [12]:
import pandas as pd

# 读入 csv
df = pd.read_csv("data/neg.csv")

# 随机抽样 5000 条
df_sample = df.sample(n=5000, random_state=42)  # random_state 保证可复现

# 保存到新文件
df_sample.to_csv("data/neg_5000.csv", index=False)

print("✅ 已保存 5000 条随机样本到 data/neg_5000.csv")

✅ 已保存 5000 条随机样本到 data/neg_5000.csv
